In [5]:
#@title **Main**
from urllib.request import urlopen
import json
import numpy as np
import pandas as pd
import os
import copy
import math
import statistics
import sklearn.metrics as metrics
 
# Evitar truncar data mostrada al usar jupyter notebook
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
 
# Constante que aloja el diccionario JSON con toda la data
DATA = None

# Obtener data JSON
if os.path.exists('./out/dataout.json'):
    DATA = json.load(open('./out/dataout.json', 'r'))
else:
    data_url = urlopen('http://nutriexcel.cl/UMDU/dataout_v2.json')
    DATA = json.loads(data_url.read())
 
# Labels base de las columnas
LABELS_BASE = {
    # Parámetros del alumno (Target)
    'p1':                            ['p1'],
    'p2':                            ['p2'],
    'np':                            ['np'],
    'p1p2':                          ['p1p2'], # Promedio p1p2 y p2p2
    'p2p2':                          ['p2p2'],
    
    # Parámetros del laboratorio (Features)
    'grade':                         ['g_lab#'],
    'attempts':                      ['a_lab#'],
    'usedtime':                      ['ut_lab#'],
    'activetime':                    ['act_lab#'],
    'disconnections':                ['dis_lab#'],      # log
    'compilationtime':               ['ct_lab#'],
    'runtimedebuggingtime':          ['rt_lab#'],
    'compilationtimeratio':          ['ctr_lab#'],
    'runtimedebuggingtimeratio':     ['rtr_lab#'],
    'errorsreductionratio':          ['err_lab#'],
    'compilationerrorsratio':        ['cer_lab#'],
    'activequartiles':               ['actq1_lab#','actq2_lab#','actq3_lab#'],
    'questionsdifficulty':           ['qd$_lab#'],
    'questionsgrades':               ['qg$_lab#'],      # Promedio
    'questionsattempts':             ['qat$_lab#'],     # Sumar - Max   # log
    'questionsactivetime':           ['qact$_lab#'],    # Promedio
    'questionsavgtime':              ['qavt$_lab#'],    # Promedio
    'questionsmaxerrors':            ['qme$_lab#'],     # Max
    'questionsmaxconsecutiveerrors': ['qmce$_lab#'],    # Max
    'questionsmaxsimilarityratio':   ['qmsr$_lab#'],    # Promedio
    'questionscorrectness':          ['qc$_lab#']       # Promedio
}
 
 
# Cantidad de preguntas por lab
LABS_LENGTHS = {
    '1': 7,
    '2': 6,
    '3': 6,
    '4': 5,
    '5': 3
}
 
def get_labels(labs:list, obj_param:list, params:list):
    '''Retorna una lista con los nombres de las columnas correspondientes.'''
    labels = []
    
    if not obj_param == None:
        for obj_p in obj_param:
            labels.extend(LABELS_BASE[obj_p])
 
    for lab in labs:
        for param in params:
            label_base = LABELS_BASE[param]
            for lb in label_base:
                aux_label = lb.replace('#', str(lab))
                if '$' in aux_label:
                    for i in range(LABS_LENGTHS[str(lab)]):
                        labels.extend([aux_label.replace('$', str(i + 1))])
                else:
                    labels.extend([aux_label])
    return labels
                    
 
def load_data(filepath = './out/dataout.json'):
    '''Retorna un diccionario importado desde un archivo JSON.'''
    return readJSON(filepath)
 
 
def convert_to_dataframe(arr:list, labels:list=None, ids_idx:list=None):
    '''Retorna un Pandas Dataframe.'''
    return pd.DataFrame(np.array(arr), columns=labels, index=ids_idx)
 
 
def get_custom_dataframe(data:dict, labs:list, obj_param:list, params:list, labels:bool=False, index:str=None):
    '''Retorna un DataFrame de Pandas con etiquetas.'''
    if params == 'all':
        params = list(LABELS_BASE.keys())[5:]
        
    data, ids_idx = get_custom_data(labs, obj_param, params, data, index)
    if labels:
        return convert_to_dataframe(data, labels=get_labels(labs, obj_param, params), ids_idx=ids_idx)
    else:
        return convert_to_dataframe(data, labels=None, ids_idx=ids_idx)
 
    
 
 
def get_custom_data(labs:list, obj_param:list, params:list, data:dict, index:str):
    dataout = []
    idxs = []
    for v1 in data['courses'].values():
        for k2,v2 in v1['students'].items():
            if index == 'id':
                idxs.append(k2)
            elif not index == None:
                idxs.append(v2[index])
            
            studentdata = []
 
            if not obj_param == None:
                for obj_p in obj_param:
                    studentdata.extend([v2[obj_p]])
            
            for labnumber in labs:
                labkey = str(list(v2['labs'].keys())[int(labnumber) - 1])
                
                for param in params:
                    val = v2['labs'][labkey][param]
                    if isinstance(val, list):
                        studentdata.extend(val)
                    else:
                        studentdata.extend([val])
            
            dataout.append(studentdata)
 
    if len(idxs) == 0:
        idxs = None
 
    return dataout, idxs
 
 
def readJSON(filename:str):
    print('Leyendo archivo: \'' + filename + '\'')
    return json.load(open(filename, 'r'))
 
# --------------------------------------------------------------------------
 
def norm_log(df):
    min_value = min(df)
    for i in range(len(df)):
        df[i] = math.log(df[i] - min_value + 1)
 
    return df
 
# --------------------------------------------------------------------------
 
def apply(df:pd.core.frame.DataFrame, columns, func, c=None, replace:bool=True):
    '''
    Aplica una función dada a una serie de parámetros dados.
    df: Dataframe.
    func: Función a aplicar (Ej.: 'sum', 'max', 'math.log', etc.).
    params: Parámetro o lista de parámetros al cual/a los cuales aplicar la función. Si el parámetro param tiene un signo '?' al final, 
            se considerarán todos aquellos que empiecen con param. Si es una lista de parámetros, cada parámetro debe ser el nombre exacto,
            es decir, no deben llevar '?'.
    c: constante o parámetro a utilizar en el cálculo (Ej: sumar constante c; log base c). Se debe omitir si no aplica.
    replace: Si es True, reemplaza la columna o columnas a las cuales se la aplica la función. Si es False, se agregan nuevas columnas.
    '''
 
    def one_param(df_new:pd.core.frame.DataFrame):
        if not columns[-1] == '?':
            try:
                aux_column = function_handler(func, df_new[columns], c)
            except KeyError:
                raise Exception('Columna ' + columns + ' no existe.')
 
            aux_column = pd.DataFrame(aux_column)
            insert_columns(df_new, aux_column, [columns], replace, func.__name__, c)
            
            return df_new
        else:
            labels = list(df_new.columns) + ['???']
            target_cols = []
            target_cols_names = []
            fg_newarr = True
            fg_endarr = False
            i = 0
 
            while i < len(labels):
                if labels[i][0] == columns[0] and columns[:-1] in labels[i]:
                    target_cols.append(i)
                    target_cols_names.append(labels[i])
                    fg_newarr = False
                elif not fg_newarr:
                    fg_endarr = True
                
                if not fg_newarr and fg_endarr:
                    fg_newarr = True
                    fg_endarr = False
                    aux_columns = []
                    
                    for row in df_new.values:
                        aux_row = [row[col] for col in target_cols]
                        aux_columns.append(function_handler(func, aux_row, c))
 
                    aux_columns = pd.DataFrame(aux_columns)
                    insert_columns(df_new, aux_columns, target_cols_names, replace, func.__name__, c)
 
                    if replace:
                        i -= len(target_cols)
                        labels = list(df_new.columns) + ['???']
 
                    target_cols = []
                    target_cols_names = []
                i += 1
            
            return df_new
 
 
    def multi_param(df_new:pd.core.frame.DataFrame):
        aux_columns = []
        target_cols = [df_new.columns.get_loc(col) for col in columns]
 
        for row in df_new.values:
            try:
                aux_row = [row[col] for col in range(len(target_cols))]
            except KeyError:
                raise Exception('Columna ' + columns + ' no existe.')
            
            aux_columns.append(function_handler(func, aux_row, c))
 
        aux_columns = pd.DataFrame(aux_columns)
        insert_columns(df_new, aux_columns, columns, replace, func.__name__, c)
 
        return df_new
 
    
    def function_handler(func, col, c=None):
        arr = copy.deepcopy(col)
        try:
            if c == None:
                return func(arr)
            else:
                return func(arr, c)
        except TypeError:
            try:
                return [function_handler(func, datum, c) for datum in arr]
            except TypeError:
                raise Exception('Operación sin sentido: x = ' + func.__name__ + '(x).')
    
 
    def insert_columns(df_new, sub_df, labels, replace, func_name, c):
        const_name = ''
 
        if not c == None:
                const_name = ',' + str(c)
 
        if len(sub_df.columns) == 1:
            merg_name = labels[0]
            
            if len(labels) > 1:
                merg_name = get_merged_name(labels[0])
 
            new_name = func_name + '(' + merg_name + const_name + ')'
 
            if replace:
                df_new[labels[0]] = sub_df
                df_new.rename(columns={labels[0]: new_name}, inplace=True)
 
                if len(labels) > 1:
                    for i in range(1, len(labels)):
                        df_new.pop(labels[i])
            else:
                df_new[new_name] = sub_df
        else:
            new_names = [func_name + '(' + cur_name + const_name + ')' for cur_name in labels]
 
            if replace:
                for i in range(len(labels)):
                    df_new[labels[i]] = sub_df[i]
                    df_new.rename(columns={labels[i]: new_names[i]}, inplace=True)
            else:
                for i in range(len(labels)):
                    df_new[new_names[i]] = sub_df[i]
 
 
    def get_merged_name(basename:str):
        merged_name = ''
        flag = True
        for i in range(len(basename)):
            if flag and basename[i].isnumeric():
                merged_name += '$'
                flag = False
            else:
                merged_name += basename[i]
 
        return merged_name
 
 
    def are_valid_column_names(columns):
        if type(columns) == list:
            for col in columns:
                if '?' in col:
                    raise Exception('Nombres de columna en lista no deben contener \'?\'.')
        
        return True
 
 
    df_new = copy.deepcopy(df)
    are_valid_column_names(columns)
    if type(columns) == list:
        return multi_param(df_new)
    else:
        return one_param(df_new)
 
 
def remove_col(df, column:str):
    if column[-1] == '?':
        for x in df.columns:
            if x[0] == column[0] and column[:-1] in x:
                df.pop(x)
    else:
        df.pop(column)
 
 
def normalise_row(df):
    df_aux = copy.deepcopy(df)
 
    for i in range(len(df_aux)):
        min_value = min(df_aux.iloc[i])
        max_value = max(df_aux.iloc[i])
 
        df_aux.iloc[i] = (df_aux.iloc[i] - min_value) / (max_value - min_value)
 
    return df_aux
 
 
def normalise_col(df, norm_params=None):
    norm_params_aux = {}
    df_aux = copy.deepcopy(df)
 
    if norm_params == None:
        for col in df_aux.columns:
            min_value = min(df_aux[col])
            max_value = max(df_aux[col])
            
            norm_params_aux[col] = {
                'min_value': min_value,
                'max_value': max_value
            }
 
            for i in range(len(df_aux[col])):
                if not max_value - min_value == 0:
                    df_aux[col].iloc[i] = (df_aux[col].iloc[i] - min_value) / (max_value - min_value)
                else:
                    df_aux[col].iloc[i] = 0
 
        return df_aux, norm_params_aux
    else:
        for col in df_aux.columns:
            for i in range(len(df_aux[col])):
                if not norm_params[col]['max_value'] - norm_params[col]['min_value'] == 0:
                    df_aux[col].iloc[i] = (df_aux[col].iloc[i] - norm_params[col]['min_value']) / (norm_params[col]['max_value'] - norm_params[col]['min_value'])
                else:
                    df_aux[col].iloc[i] = 0
        return df_aux
 
 
def normalise_log(df):
    df_aux = copy.deepcopy(df)
 
    for i in range(len(df_aux)):
        for j in range(len(df_aux.iloc[i])):
            df_aux.iloc[i][j] = math.log(df_aux.iloc[i][j]) if df_aux.iloc[i][j] > 0 else 0
 
    return df_aux

In [ ]:
print(len(datalab1_all.columns))
print(len(datalab2_all.columns))
print(len(datalab3_all.columns))
print(len(datalab4_all.columns))

In [ ]:
print(len(datalab1.columns))
print(len(datalab2.columns))
print(len(datalab3.columns))
print(len(datalab4.columns))

In [ ]:
datalab1.head()

In [6]:
#@title **Data preparation**

# Get dataframe
datalab1_all = get_custom_dataframe(DATA, [1], ['p1p2','p2p2'], 'all', labels=True, index=None)

datalab1 = copy.deepcopy(datalab1_all)
 
# Remove questionsdifficulty
remove_col(datalab1, 'qd?')
# Group columns
datalab1_all = apply(datalab1_all, ['p1p2','p2p2'], statistics.mean)
datalab1 = apply(datalab1, ['p1p2','p2p2'], statistics.mean)
datalab1 = apply(datalab1, 'dis_lab1', norm_log)
datalab1 = apply(datalab1, 'qg?', statistics.mean)
datalab1 = apply(datalab1, 'qat?', sum, replace=False)
datalab1 = apply(datalab1, 'sum(qat$_lab1)', norm_log, replace=False)
datalab1 = apply(datalab1, 'qat?', max)
datalab1 = apply(datalab1, 'qact?', statistics.mean)
datalab1 = apply(datalab1, 'qavt?', statistics.mean)
datalab1 = apply(datalab1, 'qme?', max)
datalab1 = apply(datalab1, 'qmce?', max)
datalab1 = apply(datalab1, 'qmsr?', statistics.mean)
datalab1 = apply(datalab1, 'qc?', statistics.mean)
aux = datalab1['act_lab1'] / datalab1['sum(qat$_lab1)']
for i in range(len(aux)):
    if not aux[i] > 0:
        aux[i] = 0
datalab1['avgtime_lab1'] = aux
datalab1 = datalab1.round(4)
 
 
# Get dataframe
datalab2_all = get_custom_dataframe(DATA, [2], ['p1p2','p2p2'], 'all', labels=True, index=None)
 
datalab2 = copy.deepcopy(datalab2_all)
 
# Remove questionsdifficulty
remove_col(datalab2, 'qd?')
# Group columns
datalab2_all = apply(datalab2_all, ['p1p2','p2p2'], statistics.mean)
datalab2 = apply(datalab2, ['p1p2','p2p2'], statistics.mean)
datalab2 = apply(datalab2, 'dis_lab2', norm_log)
datalab2 = apply(datalab2, 'qg?', statistics.mean)
datalab2 = apply(datalab2, 'qat?', sum, replace=False)
datalab2 = apply(datalab2, 'sum(qat$_lab2)', norm_log, replace=False)
datalab2 = apply(datalab2, 'qat?', max)
datalab2 = apply(datalab2, 'qact?', statistics.mean)
datalab2 = apply(datalab2, 'qavt?', statistics.mean)
datalab2 = apply(datalab2, 'qme?', max)
datalab2 = apply(datalab2, 'qmce?', max)
datalab2 = apply(datalab2, 'qmsr?', statistics.mean)
datalab2 = apply(datalab2, 'qc?', statistics.mean)
aux = datalab2['act_lab2'] / datalab2['sum(qat$_lab2)']
for i in range(len(aux)):
    if not aux[i] > 0:
        aux[i] = 0
datalab2['avgtime_lab2'] = aux
datalab2 = datalab2.round(4)
 
 
# Get dataframe
datalab3_all = get_custom_dataframe(DATA, [3], ['p1p2','p2p2'], 'all', labels=True, index=None)
 
datalab3 = copy.deepcopy(datalab3_all)
 
# Remove questionsdifficulty
remove_col(datalab3, 'qd?')
# Group columns
datalab3_all = apply(datalab3_all, ['p1p2','p2p2'], statistics.mean)
datalab3 = apply(datalab3, ['p1p2','p2p2'], statistics.mean)
datalab3 = apply(datalab3, 'dis_lab3', norm_log)
datalab3 = apply(datalab3, 'qg?', statistics.mean)
datalab3 = apply(datalab3, 'qat?', sum, replace=False)
datalab3 = apply(datalab3, 'sum(qat$_lab3)', norm_log, replace=False)
datalab3 = apply(datalab3, 'qat?', max)
datalab3 = apply(datalab3, 'qact?', statistics.mean)
datalab3 = apply(datalab3, 'qavt?', statistics.mean)
datalab3 = apply(datalab3, 'qme?', max)
datalab3 = apply(datalab3, 'qmce?', max)
datalab3 = apply(datalab3, 'qmsr?', statistics.mean)
datalab3 = apply(datalab3, 'qc?', statistics.mean)
aux = datalab3['act_lab3'] / datalab3['sum(qat$_lab3)']
for i in range(len(aux)):
    if not aux[i] > 0:
        aux[i] = 0
datalab3['avgtime_lab3'] = aux
datalab3 = datalab3.round(4)
 
 
# Get dataframe
datalab4_all = get_custom_dataframe(DATA, [4], ['p1p2','p2p2'], 'all', labels=True, index=None)
 
datalab4 = copy.deepcopy(datalab4_all)
 
# Remove questionsdifficulty
remove_col(datalab4, 'qd?')
# Group columns
datalab4_all = apply(datalab4_all, ['p1p2','p2p2'], statistics.mean)
datalab4 = apply(datalab4, ['p1p2','p2p2'], statistics.mean)
datalab4 = apply(datalab4, 'dis_lab4', norm_log)
datalab4 = apply(datalab4, 'qg?', statistics.mean)
datalab4 = apply(datalab4, 'qat?', sum, replace=False)
datalab4 = apply(datalab4, 'sum(qat$_lab4)', norm_log, replace=False)
datalab4 = apply(datalab4, 'qat?', max)
datalab4 = apply(datalab4, 'qact?', statistics.mean)
datalab4 = apply(datalab4, 'qavt?', statistics.mean)
datalab4 = apply(datalab4, 'qme?', max)
datalab4 = apply(datalab4, 'qmce?', max)
datalab4 = apply(datalab4, 'qmsr?', statistics.mean)
datalab4 = apply(datalab4, 'qc?', statistics.mean)
aux = datalab4['act_lab4'] / datalab4['sum(qat$_lab4)']
for i in range(len(aux)):
    if not aux[i] > 0:
        aux[i] = 0
datalab4['avgtime_lab4'] = aux
datalab4 = datalab4.round(4)

In [7]:
#@title **Parameters**

# Objective vector
TARGET = 'mean(p$p2)'
NORM_TYPE = 'col'
N_FEATURES = 5
 
 
# Import needed libraries ----------------------------------------
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
 
random_state = None # Random state for train_test_split

In [8]:
datalab1.head()

,mean(p$p2),g_lab1,a_lab1,ut_lab1,act_lab1,norm_log(dis_lab1),ct_lab1,rt_lab1,ctr_lab1,rtr_lab1,err_lab1,cer_lab1,actq1_lab1,actq2_lab1,actq3_lab1,mean(qg$_lab1),max(qat$_lab1),mean(qact$_lab1),mean(qavt$_lab1),max(qme$_lab1),max(qmce$_lab1),mean(qmsr$_lab1),mean(qc$_lab1),sum(qat$_lab1),norm_log(sum(qat$_lab1)),avgtime_lab1
0,2.00,7.0,1.0,52870.0,2144.0,1.0986,673.0,292.0,0.3139,0.1362,0.4118,0.5667,0.2096,0.2113,0.2129,1.0,11.0,137.8571,32.9762,9.0,9.0,0.8978,0.8764,30.0,3.4340,71.4667
1,2.25,7.0,1.0,12418.0,3343.0,0.6931,291.0,752.0,0.0870,0.2249,0.6471,0.4571,0.0081,0.0335,0.0357,1.0,13.0,226.0000,70.2262,4.0,4.0,0.6594,0.8201,35.0,3.5835,95.5143
2,1.25,7.0,1.0,217028.0,7613.0,2.4849,261.0,1664.0,0.0343,0.2186,0.7727,0.4364,0.5266,0.6602,0.6660,1.0,22.0,463.2857,71.4592,25.0,4.0,0.7856,0.6715,55.0,4.0254,138.4182
3,3.00,7.0,1.0,1925.0,1925.0,0.0000,337.0,1595.0,0.1751,0.8286,0.5000,0.4000,0.0342,0.0352,0.0366,1.0,6.0,288.1429,88.6286,5.0,3.0,0.9270,0.8275,25.0,3.2581,77.0000
4,2.00,7.0,1.0,4792.0,2771.0,1.3863,881.0,651.0,0.3179,0.2349,0.5294,0.5312,0.6627,0.6650,0.6678,1.0,13.0,222.0000,46.1580,9.0,9.0,0.9148,0.8259,32.0,3.4965,86.5938


In [ ]:
datalab1.head()

In [ ]:
#@title **Scale features**
 
# Scale features -------------------------------------------------
from sklearn.preprocessing import StandardScaler

scaler1 = StandardScaler()
scaler2 = StandardScaler()
scaler3 = StandardScaler()
scaler4 = StandardScaler()
scaler_all = StandardScaler()

datalab1 = datalab1[[TARGET]].join(pd.DataFrame(scaler1.fit_transform(datalab1), columns=datalab1.columns)[datalab1.columns[1:]])
datalab2 = datalab2[[TARGET]].join(pd.DataFrame(scaler2.fit_transform(datalab2), columns=datalab2.columns)[datalab2.columns[1:]])
datalab3 = datalab3[[TARGET]].join(pd.DataFrame(scaler3.fit_transform(datalab3), columns=datalab3.columns)[datalab3.columns[1:]])
datalab4 = datalab4[[TARGET]].join(pd.DataFrame(scaler4.fit_transform(datalab4), columns=datalab4.columns)[datalab4.columns[1:]])

In [ ]:
#@title **Grid/Random-SearchCV process**
 
def run_process(dataset, grid_cv, target=TARGET):
    X, y = dataset.drop(target, axis=1), np.array(dataset[target])
   
    grid_cv.fit(X,y)
 
    try:
        print('R2-test-fit:', max(grid_cv.cv_results_['mean_test_score']))
    except:
        pass

    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        grid_cv.best_estimator_.fit(X_train, y_train)
        print('R2-test', grid_cv.best_estimator_.score(X_test, y_test))
        print('MSE-test', metrics.mean_squared_error(y_test,grid_cv.best_estimator_.predict(X_test)))

        print('Best params:', grid_cv.best_params_)

        selected_features = X.columns[grid_cv.best_estimator_.steps[0][-1].get_support()]
        print('Selected features:', list(selected_features))

        return list(selected_features)
    except:
        return list(dataset.columns[1:])



---



In [ ]:
#@title **SVR - Recursive Features Elimination**

from sklearn.svm import SVR

sel_estimator = SVR(kernel='linear')
selector = RFE(sel_estimator)
estimator = SVR()

pipe = Pipeline([
    ('sel', selector),
    ('est', estimator)
])

params = {
    'sel__n_features_to_select' : [5],
    'sel__step'                 : [1,2],
    'est__C'                    : [0.01,0.1,1],
    'est__gamma'                : ['scale','auto'],
    'est__kernel'               : ['linear','poly','rbf']
}

grid_svr = GridSearchCV(estimator=pipe,
                        param_grid=params,
                        scoring='r2',
                        verbose=1,
                        n_jobs=-1,
                        return_train_score=True,
                        cv=KFold(n_splits=5, shuffle=True))

datalab1

In [ ]:
selected_features_svr_1 = run_process(datalab1,grid_svr)

In [ ]:
selected_features_svr_1 = run_process(datalab1,grid_svr)

In [ ]:
selected_features_svr_1 = run_process(datalab1,grid_svr)

In [ ]:
selected_features_svr_1 = run_process(datalab1,grid_svr)

In [ ]:
selected_features_svr_1 = run_process(datalab1,grid_svr)

In [ ]:
selected_features_svr_1 = run_process(datalab1,grid_svr)

In [ ]:
selected_features_svr_1 = run_process(datalab1,grid_svr)

In [ ]:
selected_features_svr_1 = run_process(datalab1,grid_svr)

In [ ]:
selected_features_svr_1 = run_process(datalab1,grid_svr)

In [ ]:
selected_features_svr_1 = run_process(datalab1,grid_svr)



---



datalab2

In [ ]:
selected_features_svr_2 = run_process(datalab2,grid_svr)

In [ ]:
selected_features_svr_2 = run_process(datalab2,grid_svr)

In [ ]:
selected_features_svr_2 = run_process(datalab2,grid_svr)

In [ ]:
selected_features_svr_2 = run_process(datalab2,grid_svr)

In [ ]:
selected_features_svr_2 = run_process(datalab2,grid_svr)

In [ ]:
selected_features_svr_2 = run_process(datalab2,grid_svr)

In [ ]:
selected_features_svr_2 = run_process(datalab2,grid_svr)

In [ ]:
selected_features_svr_2 = run_process(datalab2,grid_svr)

In [ ]:
selected_features_svr_2 = run_process(datalab2,grid_svr)

In [ ]:
selected_features_svr_2 = run_process(datalab2,grid_svr)



---



datalab3

In [ ]:
selected_features_svr_3 = run_process(datalab3,grid_svr)

In [ ]:
selected_features_svr_3 = run_process(datalab3,grid_svr)

In [ ]:
selected_features_svr_3 = run_process(datalab3,grid_svr)

In [ ]:
selected_features_svr_3 = run_process(datalab3,grid_svr)

In [ ]:
selected_features_svr_3 = run_process(datalab3,grid_svr)

In [ ]:
selected_features_svr_3 = run_process(datalab3,grid_svr)

In [ ]:
selected_features_svr_3 = run_process(datalab3,grid_svr)

In [ ]:
selected_features_svr_3 = run_process(datalab3,grid_svr)

In [ ]:
selected_features_svr_3 = run_process(datalab3,grid_svr)

In [ ]:
selected_features_svr_3 = run_process(datalab3,grid_svr)



---



datalab4

In [ ]:
selected_features_svr_4 = run_process(datalab4,grid_svr)

In [ ]:
selected_features_svr_4 = run_process(datalab4,grid_svr)

In [ ]:
selected_features_svr_4 = run_process(datalab4,grid_svr)

In [ ]:
selected_features_svr_4 = run_process(datalab4,grid_svr)

In [ ]:
selected_features_svr_4 = run_process(datalab4,grid_svr)

In [ ]:
selected_features_svr_4 = run_process(datalab4,grid_svr)

In [ ]:
selected_features_svr_4 = run_process(datalab4,grid_svr)

In [ ]:
selected_features_svr_4 = run_process(datalab4,grid_svr)

In [ ]:
selected_features_svr_4 = run_process(datalab4,grid_svr)

In [ ]:
selected_features_svr_4 = run_process(datalab4,grid_svr)



---



In [ ]:
estimator = SVR()

params = {
    'C'         : [0.01,0.1,1],
    'gamma'     : ['scale','auto'],
    'kernel'    : ['linear','poly','rbf']
}

grid_svr = GridSearchCV(estimator=estimator,
                        param_grid=params,
                        scoring='r2',
                        verbose=1,
                        n_jobs=-1,
                        return_train_score=True,
                        cv=KFold(n_splits=5, shuffle=True))

In [3]:
selected_features_svr_1 = ['mean(qmsr$_lab1)','actq2_lab1','actq3_lab1','mean(qg$_lab1)','actq1_lab1']
selected_features_svr_2 = ['max(qat$_lab2)','mean(qact$_lab2)','mean(qmsr$_lab2)','g_lab2','actq1_lab2']
selected_features_svr_3 = ['g_lab3','mean(qmsr$_lab3)','act_lab3','mean(qact$_lab3)','avgtime_lab3']
selected_features_svr_4 = ['max(qat$_lab4)','err_lab4','mean(qact$_lab4)','cer_lab4','rt_lab4']

datalab1_2

In [4]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [5]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [6]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [7]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [8]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [9]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [10]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [11]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [12]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [13]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined



---



datalab1_2_3

In [14]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [15]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [16]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [17]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [18]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [19]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [20]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [21]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [22]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [23]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

datalab1_2_3_4

In [24]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [25]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [26]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [27]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [28]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [29]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [30]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [31]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [32]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

In [33]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined



---



In [34]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
run_process(dataset,grid_svr)

NameError: name 'datalab1' is not defined

Plots

In [35]:
dataset = datalab1[[TARGET] + selected_features_svr_1].join(datalab2[selected_features_svr_2]).join(datalab3[selected_features_svr_3]).join(datalab4[selected_features_svr_4])
X, y = dataset.drop(TARGET, axis=1), np.array(dataset[TARGET])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
 
y_pred = grid_svr.best_estimator_.predict(X_test)

print('R2', grid_svr.score(X_test,y_test))

import matplotlib.pyplot as plt

z = [0,7]

plt.plot(y_test, y_pred, 'ro', z, z)
plt.xlabel('y_true')
plt.ylabel('y_pred')
plt.show()

NameError: name 'datalab1' is not defined



---



In [36]:
#@title **Random Forest Regressor**
 
from sklearn.ensemble import  RandomForestRegressor
 
# GradientBoostingRegressor / RandomForestRegressor / SVR(kernel='linear')
sel_estimator = GradientBoostingRegressor(random_state=random_state)
 
# RFE / SelectFromModel
selector = RFE(sel_estimator)
estimator = RandomForestRegressor(random_state=random_state, n_jobs=-1)
 
pipe = Pipeline([
    ('sel', selector),
    ('est', estimator)
])
 
params = {
    'sel__estimator__learning_rate': [0.05,0.1,0.2],
    'sel__n_features_to_select'    : [5],
    'est__n_estimators'            : [50,100,200,400],
    'est__criterion'               : ['mse','mae'],
    'est__max_features'            : ['auto','sqrt','log2']
}
 
grid_rfr = GridSearchCV(estimator=pipe,
                        param_grid=params,
                        scoring='r2',
                        verbose=1,
                        n_jobs=-1,
                        return_train_score=True,
                        cv=KFold(n_splits=5, shuffle=True))

NameError: name 'GradientBoostingRegressor' is not defined

datalab1

In [37]:
selected_features_rfr_1 = run_process(datalab1,grid_rfr)

NameError: name 'run_process' is not defined

In [38]:
selected_features_rfr_1 = run_process(datalab1,grid_rfr)

NameError: name 'run_process' is not defined

In [39]:
selected_features_rfr_1 = run_process(datalab1,grid_rfr)

NameError: name 'run_process' is not defined

In [40]:
selected_features_rfr_1 = run_process(datalab1,grid_rfr)

NameError: name 'run_process' is not defined

In [41]:
selected_features_rfr_1 = run_process(datalab1,grid_rfr)

NameError: name 'run_process' is not defined

In [42]:
selected_features_rfr_1 = run_process(datalab1,grid_rfr)

NameError: name 'run_process' is not defined

In [43]:
selected_features_rfr_1 = run_process(datalab1,grid_rfr)

NameError: name 'run_process' is not defined

In [44]:
selected_features_rfr_1 = run_process(datalab1,grid_rfr)

NameError: name 'run_process' is not defined

In [45]:
selected_features_rfr_1 = run_process(datalab1,grid_rfr)

NameError: name 'run_process' is not defined



---



datalab2

In [46]:
selected_features_rfr_2 = run_process(datalab2,grid_rfr)

NameError: name 'run_process' is not defined

In [47]:
selected_features_rfr_2 = run_process(datalab2,grid_rfr)

NameError: name 'run_process' is not defined

In [48]:
selected_features_rfr_2 = run_process(datalab2,grid_rfr)

NameError: name 'run_process' is not defined

In [49]:
selected_features_rfr_2 = run_process(datalab2,grid_rfr)

NameError: name 'run_process' is not defined

In [50]:
selected_features_rfr_2 = run_process(datalab2,grid_rfr)

NameError: name 'run_process' is not defined

In [51]:
selected_features_rfr_2 = run_process(datalab2,grid_rfr)

NameError: name 'run_process' is not defined

In [52]:
selected_features_rfr_2 = run_process(datalab2,grid_rfr)

NameError: name 'run_process' is not defined

In [53]:
selected_features_rfr_2 = run_process(datalab2,grid_rfr)

NameError: name 'run_process' is not defined

In [54]:
selected_features_rfr_2 = run_process(datalab2,grid_rfr)

NameError: name 'run_process' is not defined



---



datalab3

In [55]:
selected_features_rfr_3 = run_process(datalab3,grid_rfr)

NameError: name 'run_process' is not defined

In [56]:
selected_features_rfr_3 = run_process(datalab3,grid_rfr)

NameError: name 'run_process' is not defined

In [57]:
selected_features_rfr_3 = run_process(datalab3,grid_rfr)

NameError: name 'run_process' is not defined

In [58]:
selected_features_rfr_3 = run_process(datalab3,grid_rfr)

NameError: name 'run_process' is not defined

In [59]:
selected_features_rfr_3 = run_process(datalab3,grid_rfr)

NameError: name 'run_process' is not defined

In [60]:
selected_features_rfr_3 = run_process(datalab3,grid_rfr)

NameError: name 'run_process' is not defined

In [61]:
selected_features_rfr_3 = run_process(datalab3,grid_rfr)

NameError: name 'run_process' is not defined

In [62]:
selected_features_rfr_3 = run_process(datalab3,grid_rfr)

NameError: name 'run_process' is not defined

In [63]:
selected_features_rfr_3 = run_process(datalab3,grid_rfr)

NameError: name 'run_process' is not defined



---



datalab4

In [64]:
selected_features_rfr_4 = run_process(datalab4,grid_rfr)

NameError: name 'run_process' is not defined

In [65]:
selected_features_rfr_4 = run_process(datalab4,grid_rfr)

NameError: name 'run_process' is not defined

In [66]:
selected_features_rfr_4 = run_process(datalab4,grid_rfr)

NameError: name 'run_process' is not defined

In [67]:
selected_features_rfr_4 = run_process(datalab4,grid_rfr)

NameError: name 'run_process' is not defined

In [68]:
selected_features_rfr_4 = run_process(datalab4,grid_rfr)

NameError: name 'run_process' is not defined

In [69]:
selected_features_rfr_4 = run_process(datalab4,grid_rfr)

NameError: name 'run_process' is not defined

In [70]:
selected_features_rfr_4 = run_process(datalab4,grid_rfr)

NameError: name 'run_process' is not defined

In [ ]:
selected_features_rfr_4 = run_process(datalab4,grid_rfr)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
R2-test-fit: 0.22962021285811343
R2-test 0.12255895240539283
MSE-test 2.3181107077752965
Best params: {'est__criterion': 'mae', 'est__max_features': 'log2', 'est__n_estimators': 400, 'sel__estimator__learning_rate': 0.2, 'sel__n_features_to_select': 5}
Selected features: ['g_lab4', 'rt_lab4', 'cer_lab4', 'actq1_lab4', 'mean(qavt$_lab4)']


In [ ]:
selected_features_rfr_4 = run_process(datalab4,grid_rfr)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
R2-test-fit: 0.2328573698162033
R2-test 0.07285905085094702
MSE-test 2.344858178063425
Best params: {'est__criterion': 'mse', 'est__max_features': 'sqrt', 'est__n_estimators': 200, 'sel__estimator__learning_rate': 0.2, 'sel__n_features_to_select': 5}
Selected features: ['rt_lab4', 'rtr_lab4', 'cer_lab4', 'actq2_lab4', 'mean(qavt$_lab4)']




---



In [ ]:
estimator = RandomForestRegressor(random_state=random_state, n_jobs=-1)
 
params = {
    'n_estimators'  : [50,100,200,400],
    'criterion'     : ['mse','mae'],
    'max_features'  : ['auto','sqrt','log2']
}
 
grid_rfr = GridSearchCV(estimator=estimator,
                        param_grid=params,
                        scoring='r2',
                        verbose=1,
                        n_jobs=-1,
                        return_train_score=True,
                        cv=KFold(n_splits=5, shuffle=True))

datalab1_2

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.06604780071295309
R2-test 0.026004780335071254
MSE-test 2.524526314360119
Best params: {'criterion': 'mse', 'max_features': 'sqrt', 'n_estimators': 400}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.08627640515217602
R2-test 0.02703867022351436
MSE-test 2.5726994944196426
Best params: {'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 400}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.044900703937422626
R2-test 0.02379749008497123
MSE-test 2.814884295386905
Best params: {'criterion': 'mse', 'max_features': 'sqrt', 'n_estimators': 200}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.012245398443196432
R2-test 0.10801475843065234
MSE-test 2.0654830985863093
Best params: {'criterion': 'mae', 'max_features': 'sqrt', 'n_estimators': 400}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.07378431602592536
R2-test 0.13140791245439876
MSE-test 2.3593600357142863
Best params: {'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 200}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.09630032960042967
R2-test 0.041967912360950654
MSE-test 2.568257824939599
Best params: {'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 200}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.1015374122153107
R2-test 0.04347043552124441
MSE-test 2.1189285298226435
Best params: {'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 100}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.12087841494931492
R2-test 0.17936315562725613
MSE-test 2.1027432218915343
Best params: {'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 100}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.07467040985905342
R2-test 0.12036205639661901
MSE-test 2.622708284412203
Best params: {'criterion': 'mse', 'max_features': 'sqrt', 'n_estimators': 400}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2']

datalab1_2_3

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.147755680573684
R2-test 0.11101195775792494
MSE-test 2.1909734583333336
Best params: {'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 200}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.23312196068662475
R2-test 0.16639343504648263
MSE-test 1.9574121199776782
Best params: {'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 400}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.18954187908887468
R2-test 0.21518411061840093
MSE-test 1.856312866071429
Best params: {'criterion': 'mae', 'max_features': 'log2', 'n_estimators': 200}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.17872261930104313
R2-test 0.19659789986313458
MSE-test 2.0733521309523812
Best params: {'criterion': 'mae', 'max_features': 'log2', 'n_estimators': 50}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.16326078507732564
R2-test 0.1070529117568152
MSE-test 2.3454449285714283
Best params: {'criterion': 'mae', 'max_features': 'log2', 'n_estimators': 200}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.22979258141937947
R2-test 0.05450215164382566
MSE-test 2.1140117857142857
Best params: {'criterion': 'mae', 'max_features': 'auto', 'n_estimators': 50}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.2266959597065262
R2-test 0.17635329040331338
MSE-test 2.440279833333333
Best params: {'criterion': 'mae', 'max_features': 'log2', 'n_estimators': 50}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.1929070812033978
R2-test 0.07222849524853003
MSE-test 2.462800424525669
Best params: {'criterion': 'mae', 'max_features': 'auto', 'n_estimators': 400}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.22221137930537044
R2-test 0.185845476731321
MSE-test 2.4747632120535714
Best params: {'criterion': 'mse', 'max_features': 'sqrt', 'n_estimators': 200}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3']

datalab1_2_3_4

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3]).join(datalab4[selected_features_rfr_4])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.24157051987558145
R2-test 0.28403941560651547
MSE-test 2.0385483519345238
Best params: {'criterion': 'mse', 'max_features': 'sqrt', 'n_estimators': 200}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3',
 'rt_lab4',
 'rtr_lab4',
 'cer_lab4',
 'actq2_lab4',
 'mean(qavt$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3]).join(datalab4[selected_features_rfr_4])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.2385795855179619
R2-test 0.21906480046303567
MSE-test 1.8615210788690475
Best params: {'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 200}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3',
 'rt_lab4',
 'rtr_lab4',
 'cer_lab4',
 'actq2_lab4',
 'mean(qavt$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3]).join(datalab4[selected_features_rfr_4])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.26342413041983814
R2-test 0.23640285688539042
MSE-test 2.224716901041667
Best params: {'criterion': 'mse', 'max_features': 'sqrt', 'n_estimators': 200}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3',
 'rt_lab4',
 'rtr_lab4',
 'cer_lab4',
 'actq2_lab4',
 'mean(qavt$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3]).join(datalab4[selected_features_rfr_4])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.24383929926131126
R2-test 0.3592372346008387
MSE-test 1.6555896383928579
Best params: {'criterion': 'mse', 'max_features': 'sqrt', 'n_estimators': 400}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3',
 'rt_lab4',
 'rtr_lab4',
 'cer_lab4',
 'actq2_lab4',
 'mean(qavt$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3]).join(datalab4[selected_features_rfr_4])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.27462749202242304
R2-test 0.2864586534738772
MSE-test 1.9060561703869052
Best params: {'criterion': 'mse', 'max_features': 'sqrt', 'n_estimators': 400}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3',
 'rt_lab4',
 'rtr_lab4',
 'cer_lab4',
 'actq2_lab4',
 'mean(qavt$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3]).join(datalab4[selected_features_rfr_4])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.23339601674518978
R2-test 0.16670798002677523
MSE-test 2.1195481071428572
Best params: {'criterion': 'mse', 'max_features': 'sqrt', 'n_estimators': 50}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3',
 'rt_lab4',
 'rtr_lab4',
 'cer_lab4',
 'actq2_lab4',
 'mean(qavt$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3]).join(datalab4[selected_features_rfr_4])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.2245456730943573
R2-test 0.3165732191338049
MSE-test 1.9219487068452379
Best params: {'criterion': 'mse', 'max_features': 'sqrt', 'n_estimators': 400}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3',
 'rt_lab4',
 'rtr_lab4',
 'cer_lab4',
 'actq2_lab4',
 'mean(qavt$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3]).join(datalab4[selected_features_rfr_4])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.25409295192745696
R2-test 0.14114809115218596
MSE-test 1.8609197470238104
Best params: {'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 100}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3',
 'rt_lab4',
 'rtr_lab4',
 'cer_lab4',
 'actq2_lab4',
 'mean(qavt$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_rfr_1].join(datalab2[selected_features_rfr_2]).join(datalab3[selected_features_rfr_3]).join(datalab4[selected_features_rfr_4])
run_process(dataset,grid_rfr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
R2-test-fit: 0.22610258825938678
R2-test 0.29629842289854136
MSE-test 1.81886213374256
Best params: {'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 400}


['rtr_lab1',
 'actq1_lab1',
 'actq3_lab1',
 'mean(qmsr$_lab1)',
 'mean(qc$_lab1)',
 'ut_lab2',
 'act_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'act_lab3',
 'rt_lab3',
 'rtr_lab3',
 'cer_lab3',
 'actq2_lab3',
 'rt_lab4',
 'rtr_lab4',
 'cer_lab4',
 'actq2_lab4',
 'mean(qavt$_lab4)']



---



In [ ]:
#@title **Linear Regression**

from sklearn.linear_model import LinearRegression
 
# GradientBoostingRegressor / RandomForestRegressor / SVR(kernel='linear')
sel_estimator = GradientBoostingRegressor(random_state=random_state)
 
# RFE / SelectFromModel
selector = RFE(sel_estimator)
estimator = LinearRegression()

pipe = Pipeline([
    ('sel', selector),
    ('est', estimator)
])
 
# params = {'est__n_jobs': [-1], 
#           'est__normalize': [True], 
#           'sel__estimator__learning_rate': [0.1], 
#           'sel__estimator__n_estimators': [100], 
#           'sel__max_features': [10], 
#           'sel__prefit': [False]}

params = {
    'sel__n_features_to_select' : [5],
    'sel__step'                 : [1,2],
    'est__n_jobs'               : [-1],
}
 
grid_lr = GridSearchCV(estimator=pipe,
                       param_grid=params,
                       scoring='r2',
                       verbose=1,
                       n_jobs=-1,
                       return_train_score=True,
                       cv=KFold(n_splits=5, shuffle=True))

In [ ]:
selected_features_lr_1 = run_process(datalab1,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.5s finished


R2-test-fit: 0.02811225355053535
R2-test 0.032558596746795065
MSE-test 2.5196610989325885
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['ut_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)', 'avgtime_lab1']


In [ ]:
selected_features_lr_1 = run_process(datalab1,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
R2-test-fit: 0.03897143860883288
R2-test 0.004269003431778162
MSE-test 2.417052323166249
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['ctr_lab1', 'rtr_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)']


In [ ]:
selected_features_lr_1 = run_process(datalab1,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
R2-test-fit: 0.01836910463115362
R2-test -0.0351778738876205
MSE-test 2.087744784828418
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['act_lab1', 'rtr_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)', 'mean(qc$_lab1)']


In [ ]:
selected_features_lr_1 = run_process(datalab1,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
R2-test-fit: -0.016888356437965758
R2-test -0.072736786954678
MSE-test 2.3794821493071088
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['rtr_lab1', 'cer_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)']


In [ ]:
selected_features_lr_1 = run_process(datalab1,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
R2-test-fit: 0.031864327359388
R2-test -0.1664606626861289
MSE-test 2.7362078241077707
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['rt_lab1', 'ctr_lab1', 'cer_lab1', 'actq1_lab1', 'actq3_lab1']


In [ ]:
selected_features_lr_1 = run_process(datalab1,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
R2-test-fit: -0.015313234783943197
R2-test -0.02341772855642965
MSE-test 3.108214716784758
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['rt_lab1', 'ctr_lab1', 'actq1_lab1', 'actq3_lab1', 'avgtime_lab1']


In [ ]:
selected_features_lr_1 = run_process(datalab1,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
R2-test-fit: 0.0006082950578217616
R2-test -0.046412203405714836
MSE-test 2.188859717264865
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['rt_lab1', 'rtr_lab1', 'cer_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)']


In [ ]:
selected_features_lr_1 = run_process(datalab1,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
R2-test-fit: 0.01726029461881915
R2-test -0.161316514713018
MSE-test 2.583963808227975
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['ct_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qavt$_lab1)', 'mean(qmsr$_lab1)']


In [ ]:
selected_features_lr_1 = run_process(datalab1,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
R2-test-fit: -0.07246426008578868
R2-test 0.044227075998590326
MSE-test 2.421602954766893
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['cer_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)', 'mean(qc$_lab1)']


In [ ]:
selected_features_lr_1 = run_process(datalab1,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
R2-test-fit: 0.01283722159498144
R2-test -0.01219173166213916
MSE-test 2.573386661691931
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['ut_lab1', 'rtr_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)']




---



In [ ]:
selected_features_lr_2 = run_process(datalab2,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.8s finished


R2-test-fit: -0.018461526297947884
R2-test -0.01656735636802731
MSE-test 2.728017786168922
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['actq1_lab2', 'actq3_lab2', 'mean(qact$_lab2)', 'mean(qavt$_lab2)', 'avgtime_lab2']


In [ ]:
selected_features_lr_2 = run_process(datalab2,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.4s finished


R2-test-fit: -0.007773403567338933
R2-test 0.0021815699112528453
MSE-test 2.5931524128877563
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['rt_lab2', 'actq1_lab2', 'actq3_lab2', 'mean(qavt$_lab2)', 'sum(qat$_lab2)']


In [ ]:
selected_features_lr_2 = run_process(datalab2,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.4s finished


R2-test-fit: -0.05430646627827982
R2-test -0.004493133035586405
MSE-test 2.2068622310502883
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['actq1_lab2', 'actq3_lab2', 'mean(qact$_lab2)', 'mean(qavt$_lab2)', 'mean(qmsr$_lab2)']


In [ ]:
selected_features_lr_2 = run_process(datalab2,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.6s finished


R2-test-fit: -0.01116308943135631
R2-test -0.09358823440447739
MSE-test 2.8867796259404037
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['ct_lab2', 'rt_lab2', 'actq1_lab2', 'actq3_lab2', 'mean(qmsr$_lab2)']


In [ ]:
selected_features_lr_2 = run_process(datalab2,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.5s finished


R2-test-fit: -0.1206116023011315
R2-test -0.011551057319366675
MSE-test 2.3917258136634243
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['rtr_lab2', 'actq1_lab2', 'actq3_lab2', 'mean(qact$_lab2)', 'avgtime_lab2']


In [ ]:
selected_features_lr_2 = run_process(datalab2,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.4s finished


R2-test-fit: -0.02593232376933572
R2-test -0.0029205063247796925
MSE-test 2.783499546633971
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['ut_lab2', 'rt_lab2', 'actq2_lab2', 'actq3_lab2', 'avgtime_lab2']


In [ ]:
selected_features_lr_2 = run_process(datalab2,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.5s finished


R2-test-fit: -0.05192043074153698
R2-test -0.048880376819264715
MSE-test 2.768709730396985
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['rt_lab2', 'rtr_lab2', 'actq1_lab2', 'mean(qavt$_lab2)', 'mean(qc$_lab2)']


In [ ]:
selected_features_lr_2 = run_process(datalab2,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.6s finished


R2-test-fit: -0.0714844975712177
R2-test -0.055959025146244956
MSE-test 2.8695866093217424
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['rtr_lab2', 'actq1_lab2', 'actq3_lab2', 'mean(qavt$_lab2)', 'avgtime_lab2']


In [ ]:
selected_features_lr_2 = run_process(datalab2,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.5s finished


R2-test-fit: -0.0651951168662043
R2-test -0.01142981763075124
MSE-test 2.5371307013634965
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['ct_lab2', 'rt_lab2', 'actq3_lab2', 'mean(qavt$_lab2)', 'mean(qmsr$_lab2)']


In [ ]:
selected_features_lr_2 = run_process(datalab2,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.4s finished


R2-test-fit: 0.003965659978402014
R2-test -0.08150860925278214
MSE-test 2.5136932429176713
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['act_lab2', 'actq1_lab2', 'actq3_lab2', 'mean(qmsr$_lab2)', 'avgtime_lab2']




---



In [ ]:
selected_features_lr_3 = run_process(datalab3,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.3s finished


R2-test-fit: -0.018753894671260762
R2-test -0.036618679024586376
MSE-test 2.193674863056382
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['ut_lab3', 'act_lab3', 'rt_lab3', 'actq1_lab3', 'mean(qc$_lab3)']


In [ ]:
selected_features_lr_3 = run_process(datalab3,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.0s finished


R2-test-fit: -0.004440166709342819
R2-test 0.07991681328798839
MSE-test 2.146431435960078
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['ut_lab3', 'rt_lab3', 'actq1_lab3', 'mean(qact$_lab3)', 'avgtime_lab3']


In [ ]:
selected_features_lr_3 = run_process(datalab3,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.1s finished


R2-test-fit: 0.08125987552270511
R2-test 0.02573825965896326
MSE-test 2.117440735447757
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['ut_lab3', 'rt_lab3', 'cer_lab3', 'actq1_lab3', 'mean(qmsr$_lab3)']


In [ ]:
selected_features_lr_3 = run_process(datalab3,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.0s finished


R2-test-fit: 0.019140204136254724
R2-test 0.008802314475705342
MSE-test 2.6665279223885143
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['ut_lab3', 'act_lab3', 'rt_lab3', 'err_lab3', 'actq1_lab3']


In [ ]:
selected_features_lr_3 = run_process(datalab3,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.2s finished


R2-test-fit: 0.04813360498324746
R2-test -0.09553660911426132
MSE-test 2.561914534097449
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['ut_lab3', 'rt_lab3', 'cer_lab3', 'actq1_lab3', 'mean(qact$_lab3)']


In [ ]:
selected_features_lr_3 = run_process(datalab3,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.1s finished


R2-test-fit: 0.049884782524431494
R2-test -0.17097118403651224
MSE-test 2.7892198376738415
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['ut_lab3', 'act_lab3', 'rt_lab3', 'rtr_lab3', 'actq1_lab3']


In [ ]:
selected_features_lr_3 = run_process(datalab3,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.1s finished


R2-test-fit: 0.07901984112542626
R2-test 0.09808442468200484
MSE-test 2.466566218560531
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['rt_lab3', 'cer_lab3', 'actq1_lab3', 'mean(qavt$_lab3)', 'avgtime_lab3']


In [ ]:
selected_features_lr_3 = run_process(datalab3,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.2s finished


R2-test-fit: 0.0156938361456719
R2-test 0.24565067450876066
MSE-test 2.1057344436146046
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['g_lab3', 'rt_lab3', 'actq2_lab3', 'mean(qact$_lab3)', 'mean(qmsr$_lab3)']


In [ ]:
selected_features_lr_3 = run_process(datalab3,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.1s finished


R2-test-fit: 0.06193559239384365
R2-test 0.14678473679200843
MSE-test 2.2711414959040277
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['ut_lab3', 'rt_lab3', 'err_lab3', 'actq1_lab3', 'avgtime_lab3']


In [ ]:
selected_features_lr_3 = run_process(datalab3,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.2s finished


R2-test-fit: 0.09405118309763708
R2-test -0.0868456089935159
MSE-test 3.0897005932879864
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['ut_lab3', 'cer_lab3', 'actq1_lab3', 'max(qat$_lab3)', 'mean(qact$_lab3)']




---



In [ ]:
selected_features_lr_4 = run_process(datalab4,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.8s finished


R2-test-fit: 0.10961748103163853
R2-test -0.08406250297935403
MSE-test 2.714640536686722
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['g_lab4', 'rt_lab4', 'rtr_lab4', 'err_lab4', 'mean(qmsr$_lab4)']


In [ ]:
selected_features_lr_4 = run_process(datalab4,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.6s finished


R2-test-fit: 0.02417530486505002
R2-test 0.047913569472122
MSE-test 2.2944419405263337
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['norm_log(dis_lab4)', 'rt_lab4', 'rtr_lab4', 'err_lab4', 'actq2_lab4']


In [ ]:
selected_features_lr_4 = run_process(datalab4,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.4s finished


R2-test-fit: 0.14429264908363348
R2-test 0.17428692006586963
MSE-test 1.9188218608654248
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['g_lab4', 'rt_lab4', 'err_lab4', 'cer_lab4', 'actq1_lab4']


In [ ]:
selected_features_lr_4 = run_process(datalab4,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.4s finished


R2-test-fit: 0.04990210003970279
R2-test 0.1631059458143865
MSE-test 2.4667674636059678
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['rt_lab4', 'cer_lab4', 'actq1_lab4', 'mean(qavt$_lab4)', 'sum(qat$_lab4)']


In [ ]:
selected_features_lr_4 = run_process(datalab4,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.5s finished


R2-test-fit: 0.07181582605790733
R2-test 0.04539439920541622
MSE-test 2.4541102523239036
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['act_lab4', 'rt_lab4', 'cer_lab4', 'actq1_lab4', 'mean(qc$_lab4)']


In [ ]:
selected_features_lr_4 = run_process(datalab4,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.6s finished


R2-test-fit: 0.1378296418697363
R2-test 0.040238021597752915
MSE-test 1.9367207124060317
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['g_lab4', 'rt_lab4', 'cer_lab4', 'mean(qavt$_lab4)', 'mean(qc$_lab4)']


In [ ]:
selected_features_lr_4 = run_process(datalab4,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.5s finished


R2-test-fit: 0.17125830229583766
R2-test 0.18716368014853402
MSE-test 1.9397974171929793
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['rt_lab4', 'rtr_lab4', 'err_lab4', 'cer_lab4', 'actq1_lab4']


In [ ]:
selected_features_lr_4 = run_process(datalab4,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.5s finished


R2-test-fit: 0.15394968300004638
R2-test 0.22039316218095983
MSE-test 2.113539715414571
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['rt_lab4', 'rtr_lab4', 'err_lab4', 'cer_lab4', 'mean(qc$_lab4)']


In [ ]:
selected_features_lr_4 = run_process(datalab4,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.6s finished


R2-test-fit: -0.0032660208455336237
R2-test 0.17636202791941824
MSE-test 2.0575015829340364
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 2}
Selected features: ['g_lab4', 'act_lab4', 'actq1_lab4', 'mean(qmsr$_lab4)', 'mean(qc$_lab4)']


In [ ]:
selected_features_lr_4 = run_process(datalab4,grid_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.7s finished


R2-test-fit: 0.13889850176655444
R2-test 0.16683914167981564
MSE-test 2.084583581546277
Best params: {'est__n_jobs': -1, 'sel__n_features_to_select': 5, 'sel__step': 1}
Selected features: ['rt_lab4', 'err_lab4', 'cer_lab4', 'actq1_lab4', 'mean(qc$_lab4)']




---



In [ ]:
estimator = LinearRegression()
 
params = {
    'n_jobs'    : [-1],
}
 
grid_lr = GridSearchCV(estimator=estimator,
                       param_grid=params,
                       scoring='r2',
                       verbose=1,
                       n_jobs=-1,
                       return_train_score=True,
                       cv=KFold(n_splits=5, shuffle=True))

In [ ]:
selected_features_lr_1 = ['actq3_lab1','actq1_lab1','mean(qmsr$_lab1)','rtr_lab1','cer_lab1']
selected_features_lr_2 = ['actq3_lab2','actq1_lab2','mean(qavt$_lab2)','avgtime_lab2','rt_lab2']
selected_features_lr_3 = ['rt_lab3','actq1_lab3','ut_lab3','mean(qact$_lab3)','cer_lab3']
selected_features_lr_4 = ['rt_lab4','cer_lab4','err_lab4','actq1_lab4','mean(qc$_lab4)']

1_2

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


R2-test-fit: 0.006911971881848245
R2-test -0.036086090944452476
MSE-test 2.317345083670907
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: -0.003540234460532088
R2-test 0.0794383847029313
MSE-test 2.5780813366497703
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: -0.046590123398674944
R2-test -0.09455756729624443
MSE-test 3.0911462928259437
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.01494113936741721
R2-test -0.013487881409562696
MSE-test 2.413164056043086
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: -0.029978878483077166
R2-test -0.09356225342973579
MSE-test 2.5596454965971103
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


R2-test-fit: -0.02061660725764882
R2-test -0.2912312061511111
MSE-test 3.081248045059268
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


R2-test-fit: 0.026025901495438774
R2-test -0.011473874863089595
MSE-test 2.650700890786687
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


R2-test-fit: -0.009699739648013295
R2-test 0.08507543469984125
MSE-test 2.1607507383692717
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


R2-test-fit: -0.068743834194177
R2-test 0.008960301171420104
MSE-test 2.4510015730681665
Best params: {'n_jobs': -1}


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


R2-test-fit: 0.0009485423355833023
R2-test -0.13710154890430148
MSE-test 2.5407108704484296
Best params: {'n_jobs': -1}


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']



---



1_2_3

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.07280532021956979
R2-test 0.12141717161565224
MSE-test 2.3262559793782014
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 5.830295615796288e-05
R2-test 0.012427536060766076
MSE-test 2.4140726711615565
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.044320763610744794
R2-test 0.06863106085845805
MSE-test 2.2071133914849135
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: -0.012095135878418373
R2-test -0.10484859642135258
MSE-test 2.7897000371370266
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: -0.004509247793069604
R2-test 0.08611742188209426
MSE-test 2.382789514680097
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: -0.03789042601758361
R2-test -0.13072142250733276
MSE-test 2.775054141731726
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.036234337665218705
R2-test 0.01943093799897322
MSE-test 2.4445363669576685
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.004192329211226786
R2-test 0.016468642925129506
MSE-test 2.175581885746646
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


R2-test-fit: 0.03902897311234281
R2-test -0.06618909207438994
MSE-test 2.9331634819395886
Best params: {'n_jobs': -1}


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


R2-test-fit: 0.0742841863991344
R2-test 0.03739365033290498
MSE-test 2.413619461558209
Best params: {'n_jobs': -1}


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']



---



1_2_3_4

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3]).join(datalab4[selected_features_lr_4])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.07159460170533863
R2-test 0.17405937131044213
MSE-test 1.942184873533853
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3]).join(datalab4[selected_features_lr_4])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.13712423030689774
R2-test 0.19019282289091122
MSE-test 2.20798787288616
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3]).join(datalab4[selected_features_lr_4])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.08485515397573942
R2-test 0.17004986401493094
MSE-test 2.0482862947366565
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3]).join(datalab4[selected_features_lr_4])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.2006250060442661
R2-test 0.11536931211314372
MSE-test 2.4023173759908745
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3]).join(datalab4[selected_features_lr_4])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


R2-test-fit: 0.0876863296626134
R2-test 0.2095709912800927
MSE-test 1.6182577664266673
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3]).join(datalab4[selected_features_lr_4])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.1385503411617526
R2-test 0.1291393993575488
MSE-test 2.26177499844272
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3]).join(datalab4[selected_features_lr_4])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.11783198781579604
R2-test 0.028345362118560424
MSE-test 2.0877971158508193
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3]).join(datalab4[selected_features_lr_4])
run_process(dataset,grid_lr)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.15218574347801778
R2-test 0.1447160780650525
MSE-test 1.912049268903947
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3]).join(datalab4[selected_features_lr_4])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.19765417482762362
R2-test 0.0964875192844028
MSE-test 2.4087222095500955
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_lr_1].join(datalab2[selected_features_lr_2]).join(datalab3[selected_features_lr_3]).join(datalab4[selected_features_lr_4])
run_process(dataset,grid_lr)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
R2-test-fit: 0.13475044609413206
R2-test -0.051569875496652706
MSE-test 2.661000476068511
Best params: {'n_jobs': -1}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']



---



In [ ]:
!apt-get install swig -y
!pip install Cython numpy
!pip install auto-sklearn

In [ ]:
!pip install scikit-learn==0.23.0

In [ ]:
!pip install dask distributed

In [ ]:
#@title **ANN**

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

def create_model( nl1=1, nl2=1,  nl3=1, 
                 nn1=1000, nn2=500, nn3 = 200, lr=0.01, decay=0., l1=0.01, l2=0.01,
                act = 'relu', dropout=0, input_shape=25, output_shape=1):
    '''This is a model generating function so that we can search over neural net 
    parameters and architecture'''
    
    opt = 'SGD' # keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999,  decay=decay)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                     
    model = Sequential()
    
    # for the firt layer we need to specify the input dimensions
    first=True
    
    for i in range(nl1):
        if first:
            model.add(Dense(nn1, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn1, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    for i in range(nl2):
        if first:
            model.add(Dense(nn2, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn2, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    for i in range(nl3):
        if first:
            model.add(Dense(nn3, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn3, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    model.add(Dense(output_shape, activation='sigmoid'))
    model.compile(loss='mse', optimizer=opt, metrics=['mse'],)
    return model

# model class to use in the scikit random search CV 
model = KerasRegressor(build_fn=create_model, epochs=20, batch_size=20, verbose=1)

# learning algorithm parameters
lr=[1e-2, 1e-3, 1e-4]
decay=[1e-6,1e-9,0]

# activation
activation=['relu', 'sigmoid']

# numbers of layers
nl1 = [0,1,2,3]
nl2 = [0,1,2,3]
nl3 = [0,1,2,3]

# neurons in each layer
nn1=[20,50,100]
nn2=[10,40,70]
nn3=[5,15,30]
# nn1=[300,700,1400,2100]
# nn2=[100,400,800]
# nn3=[50,150,300]

# dropout and regularisation
dropout = [0, 0.1, 0.2, 0.3]
l1 = [0, 0.01, 0.003, 0.001, 0.0001]
l2 = [0, 0.01, 0.003, 0.001, 0.0001]

In [ ]:
# GradientBoostingRegressor / RandomForestRegressor / SVR(kernel='linear')
sel_estimator = GradientBoostingRegressor(random_state=random_state)
 
# RFE / SelectFromModel
selector = RFE(sel_estimator)
estimator = model
 
pipe = Pipeline([
    ('sel', selector),
    ('est', estimator)
])

param_grid = {
    'sel__estimator__learning_rate': [0.05,0.1,0.2],
    'sel__n_features_to_select'    : [5],
    'est__nl1':            nl1,
    'est__nl2':            nl2,
    'est__nl3':            nl3,
    'est__nn1':            nn1,
    'est__nn2':            nn2,
    'est__nn3':            nn3,
    'est__act':            activation,
    'est__l1':             l1,
    'est__l2':             l2,
    'est__lr':             lr,
    'est__decay':          decay,
    'est__dropout':        dropout, 
    'est__input_shape':    [5],
    'est__output_shape':   [1]
}

rand_cv = RandomizedSearchCV(estimator=pipe,
                             param_distributions=param_grid,
                             verbose=100,
                             n_iter=10,
                             scoring='neg_mean_squared_error',
                             n_jobs=-1,
                             return_train_score=True,
                             cv=KFold(n_splits=5, shuffle=True))

In [ ]:
run_process(datalab1, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.6826 - mse: 8.0222
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 7.7499 - mse: 7.0904
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.8642 - mse: 7.2050
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.3153 - mse: 6.6563
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 6.9455 - mse: 6.2864
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.2916 - mse: 6.6325
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.3258 - mse: 6.6668
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 7.0163 - mse: 6.3576
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1654 - mse: 6.5072
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 7.0673 - mse: 6.4096
Epoch 11

['ut_lab1', 'rtr_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qc$_lab1)']

In [ ]:
run_process(datalab1, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.0235 - mse: 7.5470
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6654 - mse: 7.1892
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.7504 - mse: 7.2746
Epoch 4/20
13/13 [==============================] - 0s 2ms/step - loss: 8.0655 - mse: 7.5900
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6406 - mse: 7.1655
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1524 - mse: 6.6774
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.2153 - mse: 6.7404
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.9910 - mse: 6.5162
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3409 - mse: 5.8663
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.7307 - mse: 6.2563
Epoch 11

['ut_lab1', 'ctr_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)']

In [ ]:
run_process(datalab1, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 992us/step - loss: 14.5028 - mse: 8.5900
Epoch 2/20
13/13 [==============================] - 0s 951us/step - loss: 14.1717 - mse: 8.3135
Epoch 3/20
13/13 [==============================] - 0s 951us/step - loss: 13.3375 - mse: 7.5308
Epoch 4/20
13/13 [==============================] - 0s 951us/step - loss: 12.5249 - mse: 6.7672
Epoch 5/20
13/13 [==============================] - 0s 909us/step - loss: 11.5566 - mse: 5.8464
Epoch 6/20
13/13 [==============================] - 0s 909us/step - loss: 12.3634 - mse: 6.7004
Epoch 7/20
13/13 [==============================] - 0s 951us/step - loss: 12.1208 - mse: 6.5061
Epoch 8/20
13/13 [==============================] - 0s 951us/step - loss: 12.3176 - mse: 6.7517
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 11.7704 - mse: 6.2540
Epoch 10/20
13/13 [==============================] - 0s 992us/step - loss: 11

['ctr_lab1', 'cer_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)']

In [ ]:
run_process(datalab1, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 992us/step - loss: 8.1453 - mse: 7.8227
Epoch 2/20
13/13 [==============================] - 0s 909us/step - loss: 7.8284 - mse: 7.5064
Epoch 3/20
13/13 [==============================] - 0s 951us/step - loss: 7.9183 - mse: 7.5970
Epoch 4/20
13/13 [==============================] - 0s 951us/step - loss: 7.2407 - mse: 6.9199
Epoch 5/20
13/13 [==============================] - 0s 951us/step - loss: 7.5546 - mse: 7.2341
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.5524 - mse: 7.2321
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.9903 - mse: 6.6701
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.5007 - mse: 6.1806
Epoch 9/20
13/13 [==============================] - 0s 951us/step - loss: 6.9146 - mse: 6.5945
Epoch 10/20
13/13 [==============================] - 0s 951us/step - loss: 6.1683 - mse: 5

['ut_lab1', 'rt_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)']

In [ ]:
run_process(datalab1, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 13.8036 - mse: 7.6704
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 12.9677 - mse: 6.8594
Epoch 3/20
13/13 [==============================] - 0s 951us/step - loss: 13.5947 - mse: 7.5105
Epoch 4/20
13/13 [==============================] - 0s 951us/step - loss: 12.7636 - mse: 6.7030
Epoch 5/20
13/13 [==============================] - 0s 909us/step - loss: 12.2728 - mse: 6.2353
Epoch 6/20
13/13 [==============================] - 0s 992us/step - loss: 11.4762 - mse: 5.4618
Epoch 7/20
13/13 [==============================] - 0s 992us/step - loss: 12.2325 - mse: 6.2410
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 11.1592 - mse: 5.1909
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 11.9898 - mse: 6.0448
Epoch 10/20
13/13 [==============================] - 0s 951us/step - loss: 11.6125 

['ctr_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)', 'sum(qat$_lab1)']

In [ ]:
run_process(datalab1, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 26.9045 - mse: 6.8973
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 26.2978 - mse: 6.6229
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 25.1946 - mse: 5.8483
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 24.9191 - mse: 5.8979
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 25.0269 - mse: 6.3275
Epoch 6/20
13/13 [==============================] - 0s 2ms/step - loss: 24.7038 - mse: 6.3232
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 23.5145 - mse: 5.4494
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 24.1944 - mse: 6.4417
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 23.5854 - mse: 6.1419
Epoch 10/20
13/13 [==============================] - 0s 2ms/step - loss: 23.0194 - mse: 5.881

['rtr_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)', 'sum(qat$_lab1)']

In [ ]:
run_process(datalab1, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.2989 - mse: 7.7322
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 8.2407 - mse: 7.6740
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.0402 - mse: 6.4736
Epoch 4/20
13/13 [==============================] - 0s 992us/step - loss: 6.9910 - mse: 6.4242
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 6.6177 - mse: 6.0509
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3925 - mse: 5.8257
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.5759 - mse: 6.0091
Epoch 8/20
13/13 [==============================] - 0s 992us/step - loss: 6.5809 - mse: 6.0142
Epoch 9/20
13/13 [==============================] - 0s 992us/step - loss: 6.8259 - mse: 6.2594
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.4306 - mse: 5.8644
Ep

['ctr_lab1', 'rtr_lab1', 'cer_lab1', 'actq1_lab1', 'actq3_lab1']

In [ ]:
run_process(datalab1, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 868us/step - loss: 9.0473 - mse: 8.0286
Epoch 2/20
13/13 [==============================] - 0s 785us/step - loss: 8.1316 - mse: 7.1155
Epoch 3/20
13/13 [==============================] - 0s 824us/step - loss: 8.2361 - mse: 7.2220
Epoch 4/20
13/13 [==============================] - 0s 992us/step - loss: 7.6148 - mse: 6.6022
Epoch 5/20
13/13 [==============================] - 0s 869us/step - loss: 7.5014 - mse: 6.4900
Epoch 6/20
13/13 [==============================] - 0s 826us/step - loss: 6.9572 - mse: 5.9475
Epoch 7/20
13/13 [==============================] - 0s 868us/step - loss: 7.1722 - mse: 6.1644
Epoch 8/20
13/13 [==============================] - 0s 785us/step - loss: 7.0817 - mse: 6.0760
Epoch 9/20
13/13 [==============================] - 0s 909us/step - loss: 7.1535 - mse: 6.1501
Epoch 10/20
13/13 [==============================] - 0s 824us/step - loss: 7.4688 - 

['rtr_lab1', 'cer_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)']

In [ ]:
run_process(datalab1, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 9.2659 - mse: 7.9594
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 8.7121 - mse: 7.4074
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 9.1714 - mse: 7.8686
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 8.6660 - mse: 7.3647
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.8996 - mse: 6.5998
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.9327 - mse: 6.6341
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.4676 - mse: 6.1703
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 7.4870 - mse: 6.1909
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 8.2493 - mse: 6.9546
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1754 - mse: 5.8821
Epoch 11

['ut_lab1', 'ct_lab1', 'rt_lab1', 'actq1_lab1', 'actq3_lab1']

In [ ]:
run_process(datalab1, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 2ms/step - loss: 8.4635 - mse: 8.0557
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 7.5303 - mse: 7.1229
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.9291 - mse: 7.5219
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.0788 - mse: 6.6718
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.4540 - mse: 7.0473
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.6036 - mse: 6.1970
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8894 - mse: 6.4830
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.4861 - mse: 6.0798
Epoch 9/20
13/13 [==============================] - 0s 2ms/step - loss: 6.3200 - mse: 5.9139
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.5198 - mse: 6.1139
Epoch 11

['rt_lab1', 'rtr_lab1', 'actq1_lab1', 'actq3_lab1', 'mean(qmsr$_lab1)']



---



In [ ]:
run_process(datalab2, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 9.7164 - mse: 8.0854
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 9.2435 - mse: 7.6148
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 9.3458 - mse: 7.7190
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 9.0455 - mse: 7.4203
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 8.3047 - mse: 6.6810
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 8.0497 - mse: 6.4274
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.8085 - mse: 6.1876
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 7.5675 - mse: 5.9479
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 8.4962 - mse: 6.8782
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 8.0586 - mse: 6.4420
Epoch 11

['ut_lab2', 'actq1_lab2', 'actq3_lab2', 'mean(qact$_lab2)', 'avgtime_lab2']

In [ ]:
run_process(datalab2, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.6815 - mse: 7.8491
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 8.5542 - mse: 7.7229
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 8.3731 - mse: 7.5427
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 8.5360 - mse: 7.7062
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.7001 - mse: 6.8709
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.2462 - mse: 6.4173
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.7133 - mse: 6.8848
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1896 - mse: 6.3617
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.5777 - mse: 5.7504
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8789 - mse: 6.0523
Epoch 11

['ut_lab2', 'ctr_lab2', 'actq1_lab2', 'actq3_lab2', 'mean(qmsr$_lab2)']

In [ ]:
run_process(datalab2, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 13.0324 - mse: 7.9289
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 12.2713 - mse: 7.1871
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 11.7341 - mse: 6.6692
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 12.3579 - mse: 7.3120
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 11.1865 - mse: 6.1597
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 11.9154 - mse: 6.9076
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 11.3946 - mse: 6.4057
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 11.1967 - mse: 6.2267
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 11.8205 - mse: 6.8693
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 11.5128 - mse: 6.580

['act_lab2', 'actq1_lab2', 'actq3_lab2', 'mean(qavt$_lab2)', 'avgtime_lab2']

In [ ]:
run_process(datalab2, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 9.6900 - mse: 7.7917
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 10.0924 - mse: 8.1979
Epoch 3/20
13/13 [==============================] - 0s 2ms/step - loss: 8.1642 - mse: 6.2734
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 8.8785 - mse: 6.9913
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 8.5492 - mse: 6.6655
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1966 - mse: 5.3164
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.8801 - mse: 6.0033
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 8.4710 - mse: 6.5977
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 7.7209 - mse: 5.8511
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 8.2796 - mse: 6.4134
Epoch 1

['actq1_lab2',
 'actq3_lab2',
 'mean(qact$_lab2)',
 'max(qme$_lab2)',
 'avgtime_lab2']

In [ ]:
run_process(datalab2, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 11.6526 - mse: 8.2571
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 11.4081 - mse: 8.0369
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 10.1666 - mse: 6.8189
Epoch 4/20
13/13 [==============================] - 0s 992us/step - loss: 10.1835 - mse: 6.8577
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 10.0776 - mse: 6.7731
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 9.9152 - mse: 6.6316
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 10.1441 - mse: 6.8815
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 9.2364 - mse: 5.9950
Epoch 9/20
13/13 [==============================] - 0s 992us/step - loss: 9.5996 - mse: 6.3797
Epoch 10/20
13/13 [==============================] - 0s 951us/step - loss: 9.8825 - mse: 6.6

['ut_lab2', 'rtr_lab2', 'actq1_lab2', 'actq3_lab2', 'mean(qavt$_lab2)']

In [ ]:
run_process(datalab2, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 0s 909us/step - loss: 8.0135 - mse: 8.0135
Epoch 2/20
13/13 [==============================] - 0s 868us/step - loss: 7.4412 - mse: 7.4412
Epoch 3/20
13/13 [==============================] - 0s 868us/step - loss: 6.4651 - mse: 6.4651
Epoch 4/20
13/13 [==============================] - 0s 827us/step - loss: 5.9376 - mse: 5.9376
Epoch 5/20
13/13 [==============================] - 0s 868us/step - loss: 6.1402 - mse: 6.1402
Epoch 6/20
13/13 [==============================] - 0s 951us/step - loss: 6.3928 - mse: 6.3928
Epoch 7/20
13/13 [==============================] - 0s 909us/step - loss: 5.9053 - mse: 5.9053
Epoch 8/20
13/13 [==============================] - 0s 868us/step - loss: 6.1373 - mse: 6.1373
Epoch 9/20
13/13 [==============================] - 0s 868us/step - loss: 5.7555 - mse: 5.7555
Epoch 10/20
13/13 [==============================] - 0s 868us/step - loss: 6.3118 - 

['ct_lab2', 'rtr_lab2', 'actq1_lab2', 'actq3_lab2', 'avgtime_lab2']

In [ ]:
run_process(datalab2, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 785us/step - loss: 8.5729 - mse: 8.0444
Epoch 2/20
13/13 [==============================] - 0s 827us/step - loss: 8.0515 - mse: 7.5229
Epoch 3/20
13/13 [==============================] - 0s 785us/step - loss: 7.6311 - mse: 7.1021
Epoch 4/20
13/13 [==============================] - 0s 826us/step - loss: 7.0794 - mse: 6.5500
Epoch 5/20
13/13 [==============================] - 0s 785us/step - loss: 6.6926 - mse: 6.1629
Epoch 6/20
13/13 [==============================] - 0s 785us/step - loss: 6.9186 - mse: 6.3888
Epoch 7/20
13/13 [==============================] - 0s 785us/step - loss: 6.7598 - mse: 6.2299
Epoch 8/20
13/13 [==============================] - 0s 827us/step - loss: 6.3481 - mse: 5.8184
Epoch 9/20
13/13 [==============================] - 0s 826us/step - loss: 6.7715 - mse: 6.2420
Epoch 10/20
13/13 [==============================] - 0s 827us/step - loss: 6.3876 - 

['rt_lab2', 'actq2_lab2', 'actq3_lab2', 'max(qat$_lab2)', 'avgtime_lab2']

In [ ]:
run_process(datalab2, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 11.4448 - mse: 9.6790
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 9.3968 - mse: 7.6331
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 8.1757 - mse: 6.4142
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.5807 - mse: 5.8214
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.9994 - mse: 6.2424
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.9374 - mse: 6.1826
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.5665 - mse: 5.8140
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 8.1194 - mse: 6.3692
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 7.7749 - mse: 6.0270
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 7.9993 - mse: 6.2536
Epoch 1

['ct_lab2', 'err_lab2', 'actq1_lab2', 'mean(qavt$_lab2)', 'avgtime_lab2']

In [ ]:
run_process(datalab2, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.0855 - mse: 8.0651
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 7.2131 - mse: 7.1926
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8374 - mse: 6.8170
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.0756 - mse: 7.0552
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 6.9103 - mse: 6.8898
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.1078 - mse: 6.0873
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 5.9413 - mse: 5.9207
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.9252 - mse: 6.9046
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2041 - mse: 6.1835
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.5090 - mse: 6.4884
Epoch 11

['act_lab2', 'rtr_lab2', 'actq1_lab2', 'mean(qavt$_lab2)', 'avgtime_lab2']

In [ ]:
run_process(datalab2, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 7.6416 - mse: 7.4683
Epoch 2/20
13/13 [==============================] - 0s 992us/step - loss: 7.9040 - mse: 7.7309
Epoch 3/20
13/13 [==============================] - 0s 909us/step - loss: 7.6225 - mse: 7.4495
Epoch 4/20
13/13 [==============================] - 0s 909us/step - loss: 7.0056 - mse: 6.8325
Epoch 5/20
13/13 [==============================] - 0s 992us/step - loss: 7.2018 - mse: 7.0286
Epoch 6/20
13/13 [==============================] - 0s 951us/step - loss: 7.1146 - mse: 6.9412
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2087 - mse: 6.0351
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3847 - mse: 6.2109
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2219 - mse: 6.0480
Epoch 10/20
13/13 [==============================] - 0s 992us/step - loss: 6.2416 - mse: 6.0

['ut_lab2', 'act_lab2', 'actq3_lab2', 'mean(qmsr$_lab2)', 'avgtime_lab2']



---



In [ ]:
run_process(datalab3, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 6.6505 - mse: 5.9281
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3307 - mse: 5.6094
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2318 - mse: 5.5116
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 6.7077 - mse: 5.9886
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.0959 - mse: 6.3780
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3541 - mse: 5.6372
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3149 - mse: 5.5991
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.4298 - mse: 5.7152
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3042 - mse: 5.5907
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8140 - mse: 6.1015
Epoch 11

['ut_lab3', 'act_lab3', 'rt_lab3', 'actq2_lab3', 'avgtime_lab3']

In [ ]:
run_process(datalab3, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 9.7564 - mse: 8.1415
Epoch 2/20
13/13 [==============================] - 0s 992us/step - loss: 9.1375 - mse: 7.5269
Epoch 3/20
13/13 [==============================] - 0s 951us/step - loss: 9.3191 - mse: 7.7124
Epoch 4/20
13/13 [==============================] - 0s 951us/step - loss: 8.6042 - mse: 7.0014
Epoch 5/20
13/13 [==============================] - 0s 909us/step - loss: 9.1984 - mse: 7.5992
Epoch 6/20
13/13 [==============================] - 0s 992us/step - loss: 8.5521 - mse: 6.9563
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 8.0968 - mse: 6.5043
Epoch 8/20
13/13 [==============================] - 0s 951us/step - loss: 7.7293 - mse: 6.1401
Epoch 9/20
13/13 [==============================] - 0s 909us/step - loss: 7.8610 - mse: 6.2752
Epoch 10/20
13/13 [==============================] - 0s 909us/step - loss: 7.4343 - mse:

['act_lab3', 'rt_lab3', 'cer_lab3', 'mean(qc$_lab3)', 'avgtime_lab3']

In [ ]:
run_process(datalab3, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.2358 - mse: 7.2775
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 9.2127 - mse: 8.2549
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.3077 - mse: 6.3500
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1237 - mse: 6.1659
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1395 - mse: 6.1818
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8770 - mse: 5.9196
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.6694 - mse: 5.7125
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.7458 - mse: 5.7895
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 7.4749 - mse: 6.5193
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1886 - mse: 6.2337
Epoch 11

['g_lab3', 'ut_lab3', 'act_lab3', 'actq3_lab3', 'avgtime_lab3']

In [ ]:
run_process(datalab3, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 951us/step - loss: 7.5350 - mse: 7.4948
Epoch 2/20
13/13 [==============================] - 0s 909us/step - loss: 6.6870 - mse: 6.6468
Epoch 3/20
13/13 [==============================] - 0s 868us/step - loss: 6.6275 - mse: 6.5872
Epoch 4/20
13/13 [==============================] - 0s 992us/step - loss: 6.0386 - mse: 5.9984
Epoch 5/20
13/13 [==============================] - 0s 951us/step - loss: 5.7779 - mse: 5.7377
Epoch 6/20
13/13 [==============================] - 0s 910us/step - loss: 6.1536 - mse: 6.1134
Epoch 7/20
13/13 [==============================] - 0s 868us/step - loss: 5.9528 - mse: 5.9126
Epoch 8/20
13/13 [==============================] - 0s 951us/step - loss: 6.0097 - mse: 5.9695
Epoch 9/20
13/13 [==============================] - 0s 953us/step - loss: 6.3375 - mse: 6.2973
Epoch 10/20
13/13 [==============================] - 0s 953us/step - loss: 5.7568 - 

['ut_lab3', 'act_lab3', 'rt_lab3', 'actq1_lab3', 'sum(qat$_lab3)']

In [ ]:
run_process(datalab3, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 9.0973 - mse: 9.0632
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6704 - mse: 7.6363
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.3309 - mse: 7.2968
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.0885 - mse: 7.0544
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.5392 - mse: 7.5051
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.9255 - mse: 6.8914
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.4744 - mse: 6.4403
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.0471 - mse: 6.0130
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.5862 - mse: 6.5520
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2509 - mse: 6.2167
Epoch 11

['ut_lab3', 'act_lab3', 'rt_lab3', 'actq1_lab3', 'actq3_lab3']

In [ ]:
run_process(datalab3, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 7.4865 - mse: 7.4113
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 6.5437 - mse: 6.4684
Epoch 3/20
13/13 [==============================] - 0s 951us/step - loss: 6.8114 - mse: 6.7360
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 6.7508 - mse: 6.6752
Epoch 5/20
13/13 [==============================] - 0s 951us/step - loss: 6.0499 - mse: 5.9744
Epoch 6/20
13/13 [==============================] - 0s 909us/step - loss: 6.4969 - mse: 6.4212
Epoch 7/20
13/13 [==============================] - 0s 951us/step - loss: 6.2912 - mse: 6.2155
Epoch 8/20
13/13 [==============================] - 0s 909us/step - loss: 5.8991 - mse: 5.8233
Epoch 9/20
13/13 [==============================] - 0s 992us/step - loss: 6.2640 - mse: 6.1882
Epoch 10/20
13/13 [==============================] - 0s 951us/step - loss: 6.1726 - mse: 6

['ut_lab3', 'rt_lab3', 'cer_lab3', 'actq2_lab3', 'avgtime_lab3']

In [ ]:
run_process(datalab3, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.8261 - mse: 8.5529
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 7.7783 - mse: 7.5051
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8972 - mse: 6.6238
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3770 - mse: 6.1036
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 6.0644 - mse: 5.7908
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.0930 - mse: 5.8193
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8745 - mse: 6.6008
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.0359 - mse: 5.7622
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.0238 - mse: 5.7501
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3791 - mse: 6.1054
Epoch 11

['ut_lab3', 'rt_lab3', 'cer_lab3', 'actq1_lab3', 'mean(qact$_lab3)']

In [ ]:
run_process(datalab3, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 9.2023 - mse: 7.5769
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 9.3435 - mse: 7.7196
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 8.2195 - mse: 6.5967
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 8.0212 - mse: 6.3995
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 8.1078 - mse: 6.4870
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 8.3410 - mse: 6.7214
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.9257 - mse: 6.3074
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6742 - mse: 6.0572
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 7.7268 - mse: 6.1113
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6806 - mse: 6.0667
Epoch 11

['g_lab3', 'ut_lab3', 'act_lab3', 'rtr_lab3', 'actq1_lab3']

In [ ]:
run_process(datalab3, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 7.5737 - mse: 7.5096
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 6.6554 - mse: 6.5913
Epoch 3/20
13/13 [==============================] - 0s 992us/step - loss: 5.8053 - mse: 5.7412
Epoch 4/20
13/13 [==============================] - 0s 951us/step - loss: 5.8321 - mse: 5.7680
Epoch 5/20
13/13 [==============================] - 0s 909us/step - loss: 5.7333 - mse: 5.6692
Epoch 6/20
13/13 [==============================] - 0s 909us/step - loss: 6.2460 - mse: 6.1819
Epoch 7/20
13/13 [==============================] - 0s 909us/step - loss: 6.2375 - mse: 6.1734
Epoch 8/20
13/13 [==============================] - 0s 992us/step - loss: 5.8460 - mse: 5.7819
Epoch 9/20
13/13 [==============================] - 0s 992us/step - loss: 6.2869 - mse: 6.2228
Epoch 10/20
13/13 [==============================] - 0s 951us/step - loss: 6.2786 - mse:

['ut_lab3', 'cer_lab3', 'actq1_lab3', 'mean(qact$_lab3)', 'avgtime_lab3']

In [ ]:
run_process(datalab3, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 7.9715 - mse: 7.8836
Epoch 2/20
13/13 [==============================] - 0s 992us/step - loss: 8.1601 - mse: 8.0722
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.8127 - mse: 7.7248
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8994 - mse: 6.8115
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 6.6208 - mse: 6.5328
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.4551 - mse: 6.3671
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.7747 - mse: 6.6866
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.4908 - mse: 6.4028
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 5.6177 - mse: 5.5296
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 5.8308 - mse: 5.7427
Epoch 

['g_lab3', 'ut_lab3', 'act_lab3', 'actq2_lab3', 'actq3_lab3']



---



In [ ]:
run_process(datalab4, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.2111 - mse: 8.0788
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6402 - mse: 7.5079
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1245 - mse: 6.9921
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.0782 - mse: 6.9455
Epoch 5/20
13/13 [==============================] - 0s 951us/step - loss: 6.6990 - mse: 6.5660
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.4723 - mse: 6.3390
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.6686 - mse: 6.5350
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2658 - mse: 6.1321
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.5876 - mse: 6.4537
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2314 - mse: 6.0973
Epoch 

['ut_lab4', 'rt_lab4', 'rtr_lab4', 'cer_lab4', 'mean(qc$_lab4)']

In [ ]:
run_process(datalab4, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 909us/step - loss: 8.5769 - mse: 7.8352
Epoch 2/20
13/13 [==============================] - 0s 909us/step - loss: 8.3093 - mse: 7.5671
Epoch 3/20
13/13 [==============================] - 0s 992us/step - loss: 7.7028 - mse: 6.9600
Epoch 4/20
13/13 [==============================] - 0s 910us/step - loss: 7.5920 - mse: 6.8489
Epoch 5/20
13/13 [==============================] - 0s 909us/step - loss: 7.7236 - mse: 6.9807
Epoch 6/20
13/13 [==============================] - 0s 909us/step - loss: 6.8561 - mse: 6.1139
Epoch 7/20
13/13 [==============================] - 0s 909us/step - loss: 7.0890 - mse: 6.3477
Epoch 8/20
13/13 [==============================] - 0s 951us/step - loss: 6.7793 - mse: 6.0390
Epoch 9/20
13/13 [==============================] - 0s 909us/step - loss: 7.1415 - mse: 6.4024
Epoch 10/20
13/13 [==============================] - 0s 909us/step - loss: 6.7983 - 

['rtr_lab4', 'cer_lab4', 'actq1_lab4', 'mean(qavt$_lab4)', 'mean(qc$_lab4)']

In [ ]:
run_process(datalab4, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 909us/step - loss: 11.9637 - mse: 7.9846
Epoch 2/20
13/13 [==============================] - 0s 951us/step - loss: 11.4999 - mse: 7.5325
Epoch 3/20
13/13 [==============================] - 0s 909us/step - loss: 10.3163 - mse: 6.3599
Epoch 4/20
13/13 [==============================] - 0s 868us/step - loss: 10.2902 - mse: 6.3452
Epoch 5/20
13/13 [==============================] - 0s 909us/step - loss: 10.7631 - mse: 6.8299
Epoch 6/20
13/13 [==============================] - 0s 909us/step - loss: 10.2656 - mse: 6.3448
Epoch 7/20
13/13 [==============================] - 0s 950us/step - loss: 9.9204 - mse: 6.0124
Epoch 8/20
13/13 [==============================] - 0s 868us/step - loss: 9.7473 - mse: 5.8524
Epoch 9/20
13/13 [==============================] - 0s 909us/step - loss: 9.9459 - mse: 6.0642
Epoch 10/20
13/13 [==============================] - 0s 909us/step - loss: 10.

['act_lab4', 'rtr_lab4', 'cer_lab4', 'mean(qg$_lab4)', 'mean(qc$_lab4)']

In [ ]:
run_process(datalab4, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 951us/step - loss: 10.1468 - mse: 7.0523
Epoch 2/20
13/13 [==============================] - 0s 909us/step - loss: 9.2386 - mse: 6.1662
Epoch 3/20
13/13 [==============================] - 0s 909us/step - loss: 8.4921 - mse: 5.4423
Epoch 4/20
13/13 [==============================] - 0s 868us/step - loss: 8.9969 - mse: 5.9697
Epoch 5/20
13/13 [==============================] - 0s 909us/step - loss: 8.9031 - mse: 5.8986
Epoch 6/20
13/13 [==============================] - 0s 868us/step - loss: 8.2910 - mse: 5.3092
Epoch 7/20
13/13 [==============================] - 0s 951us/step - loss: 8.4034 - mse: 5.4443
Epoch 8/20
13/13 [==============================] - 0s 951us/step - loss: 8.8979 - mse: 5.9614
Epoch 9/20
13/13 [==============================] - 0s 827us/step - loss: 9.2299 - mse: 6.3159
Epoch 10/20
13/13 [==============================] - 0s 912us/step - loss: 8.8333 -

['act_lab4', 'rt_lab4', 'rtr_lab4', 'actq1_lab4', 'mean(qc$_lab4)']

In [ ]:
run_process(datalab4, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 7.9742 - mse: 7.8804
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 8.2200 - mse: 8.1262
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.3984 - mse: 7.3045
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.8059 - mse: 7.7120
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1150 - mse: 7.0211
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8126 - mse: 6.7186
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 5.9597 - mse: 5.8656
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 7.2368 - mse: 7.1426
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2455 - mse: 6.1513
Epoch 10/20
13/13 [==============================] - 0s 992us/step - loss: 6.6152 - mse: 6.5210
Epoch 

['g_lab4', 'rt_lab4', 'rtr_lab4', 'actq1_lab4', 'mean(qmsr$_lab4)']

In [ ]:
run_process(datalab4, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.1951 - mse: 7.7119
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 8.2501 - mse: 7.7670
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.2426 - mse: 6.7593
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8450 - mse: 6.3616
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1633 - mse: 6.6797
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.1359 - mse: 5.6522
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.7480 - mse: 6.2643
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.5294 - mse: 6.0457
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 5.7020 - mse: 5.2184
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.1966 - mse: 5.7131
Epoch 11

['norm_log(dis_lab4)', 'rt_lab4', 'cer_lab4', 'actq1_lab4', 'mean(qavt$_lab4)']

In [ ]:
run_process(datalab4, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 23.0075 - mse: 8.4800
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 20.7132 - mse: 6.3627
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 20.9366 - mse: 6.7611
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 20.0717 - mse: 6.0701
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 19.9190 - mse: 6.0900
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 19.5214 - mse: 5.8642
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 19.8189 - mse: 6.3324
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 19.6589 - mse: 6.3422
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 19.0831 - mse: 5.9350
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 18.9316 - mse: 5.951

['rt_lab4', 'cer_lab4', 'actq1_lab4', 'mean(qavt$_lab4)', 'mean(qc$_lab4)']

In [ ]:
run_process(datalab4, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 827us/step - loss: 7.3647 - mse: 7.3647
Epoch 2/20
13/13 [==============================] - 0s 827us/step - loss: 6.9304 - mse: 6.9304
Epoch 3/20
13/13 [==============================] - 0s 909us/step - loss: 6.1387 - mse: 6.1387
Epoch 4/20
13/13 [==============================] - 0s 827us/step - loss: 6.1378 - mse: 6.1378
Epoch 5/20
13/13 [==============================] - 0s 992us/step - loss: 6.3307 - mse: 6.3307
Epoch 6/20
13/13 [==============================] - 0s 951us/step - loss: 6.6716 - mse: 6.6716
Epoch 7/20
13/13 [==============================] - 0s 951us/step - loss: 6.1432 - mse: 6.1432
Epoch 8/20
13/13 [==============================] - 0s 868us/step - loss: 5.8555 - mse: 5.8555
Epoch 9/20
13/13 [==============================] - 0s 827us/step - loss: 5.8189 - mse: 5.8189
Epoch 10/20
13/13 [==============================] - 0s 951us/step - loss: 6.4217 - 

['rt_lab4', 'rtr_lab4', 'err_lab4', 'actq2_lab4', 'mean(qc$_lab4)']

In [ ]:
run_process(datalab4, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 11.1054 - mse: 8.5651
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 10.1554 - mse: 7.6272
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 9.2997 - mse: 6.7826
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 8.5836 - mse: 6.0767
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 9.4489 - mse: 6.9520
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 8.6434 - mse: 6.1567
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 8.6997 - mse: 6.2234
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 8.3749 - mse: 5.9095
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 8.7071 - mse: 6.2528
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 8.4470 - mse: 6.0040
Epoch 

['rt_lab4', 'rtr_lab4', 'cer_lab4', 'actq1_lab4', 'mean(qc$_lab4)']

In [ ]:
run_process(datalab4, rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 9.8232 - mse: 8.5742
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 9.1387 - mse: 7.8910
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 8.6030 - mse: 7.3561
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.3176 - mse: 6.0713
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.0710 - mse: 5.8251
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.9138 - mse: 6.6684
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6337 - mse: 6.3889
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8303 - mse: 5.5863
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6414 - mse: 6.3982
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.7945 - mse: 5.5522
Epoch 11

['rt_lab4', 'err_lab4', 'cer_lab4', 'actq1_lab4', 'mean(qc$_lab4)']

In [ ]:
estimator = model

param_grid = {
    'nl1':            nl1,
    'nl2':            nl2,
    'nl3':            nl3,
    'nn1':            nn1,
    'nn2':            nn2,
    'nn3':            nn3,
    'act':            activation,
    'l1':             l1,
    'l2':             l2,
    'lr':             lr,
    'decay':          decay,
    'dropout':        dropout, 
    'input_shape':    [20],
    'output_shape':   [1]
}

rand_cv = RandomizedSearchCV(estimator=estimator,
                             param_distributions=param_grid,
                             verbose=100,
                             n_iter=10,
                             scoring='neg_mean_squared_error',
                             n_jobs=-1,
                             return_train_score=True,
                             cv=KFold(n_splits=5, shuffle=True))

In [ ]:
selected_features_ann_1 = ['actq3_lab1','actq1_lab1','mean(qmsr$_lab1)','rtr_lab1','cer_lab1']
selected_features_ann_2 = ['actq3_lab2','actq1_lab2','mean(qavt$_lab2)','avgtime_lab2','rt_lab2']
selected_features_ann_3 = ['rt_lab3','actq1_lab3','ut_lab3','mean(qact$_lab3)','cer_lab3']
selected_features_ann_4 = ['rt_lab4','cer_lab4','err_lab4','actq1_lab4','mean(qc$_lab4)']

1_2

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 7.9412 - mse: 7.6026
Epoch 2/20
13/13 [==============================] - 0s 951us/step - loss: 7.4373 - mse: 7.0986
Epoch 3/20
13/13 [==============================] - 0s 992us/step - loss: 6.7720 - mse: 6.4328
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 5.9585 - mse: 5.6190
Epoch 5/20
13/13 [==============================] - 0s 951us/step - loss: 6.0052 - mse: 5.6653
Epoch 6/20
13/13 [==============================] - 0s 951us/step - loss: 5.8995 - mse: 5.5595
Epoch 7/20
13/13 [==============================] - 0s 909us/step - loss: 6.7229 - mse: 6.3829
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 7.3314 - mse: 6.9913
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8577 - mse: 6.5176
Epoch 10/20
13/13 [==============================] - 0s 992us/step - loss: 6.4638 - mse: 6.1

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.8065 - mse: 8.0262
Epoch 2/20
13/13 [==============================] - 0s 951us/step - loss: 7.0924 - mse: 6.3132
Epoch 3/20
13/13 [==============================] - 0s 951us/step - loss: 6.6397 - mse: 5.8616
Epoch 4/20
13/13 [==============================] - 0s 950us/step - loss: 6.3780 - mse: 5.6009
Epoch 5/20
13/13 [==============================] - 0s 868us/step - loss: 7.1986 - mse: 6.4226
Epoch 6/20
13/13 [==============================] - 0s 868us/step - loss: 6.6423 - mse: 5.8674
Epoch 7/20
13/13 [==============================] - 0s 909us/step - loss: 6.5313 - mse: 5.7574
Epoch 8/20
13/13 [==============================] - 0s 909us/step - loss: 6.3361 - mse: 5.5633
Epoch 9/20
13/13 [==============================] - 0s 909us/step - loss: 6.8393 - mse: 6.0676
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.7274 - mse:

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 7.6977 - mse: 7.5564
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 8.1878 - mse: 8.0463
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8323 - mse: 6.6904
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 6.0625 - mse: 5.9201
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3927 - mse: 6.2501
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 5.5243 - mse: 5.3814
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.1783 - mse: 6.0353
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.1278 - mse: 5.9847
Epoch 9/20
13/13 [==============================] - 0s 951us/step - loss: 6.1442 - mse: 6.0010
Epoch 10/20
13/13 [==============================] - 0s 992us/step - loss: 5.4358 - mse: 5.2926
Epoc

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6192 - mse: 7.5016
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 6.4630 - mse: 6.3455
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2530 - mse: 6.1356
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 5.9422 - mse: 5.8248
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 6.1411 - mse: 6.0238
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.0606 - mse: 5.9433
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 5.9040 - mse: 5.7868
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 5.8924 - mse: 5.7752
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 5.8770 - mse: 5.7598
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 5.2410 - mse: 5.1240
Epoch 11

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 951us/step - loss: 8.6540 - mse: 8.3376
Epoch 2/20
13/13 [==============================] - 0s 909us/step - loss: 8.0978 - mse: 7.7809
Epoch 3/20
13/13 [==============================] - 0s 992us/step - loss: 7.3226 - mse: 7.0051
Epoch 4/20
13/13 [==============================] - 0s 868us/step - loss: 6.2479 - mse: 5.9297
Epoch 5/20
13/13 [==============================] - 0s 951us/step - loss: 6.1541 - mse: 5.8355
Epoch 6/20
13/13 [==============================] - 0s 951us/step - loss: 5.9330 - mse: 5.6142
Epoch 7/20
13/13 [==============================] - 0s 909us/step - loss: 6.1262 - mse: 5.8074
Epoch 8/20
13/13 [==============================] - 0s 909us/step - loss: 6.1515 - mse: 5.8328
Epoch 9/20
13/13 [==============================] - 0s 909us/step - loss: 6.0689 - mse: 5.7503
Epoch 10/20
13/13 [==============================] - 0s 868us/step - loss: 6.5633 - 

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 11.4348 - mse: 7.5510
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 10.5614 - mse: 6.6924
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 10.3793 - mse: 6.5253
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 9.6056 - mse: 5.7667
Epoch 5/20
13/13 [==============================] - 0s 951us/step - loss: 9.4930 - mse: 5.6691
Epoch 6/20
13/13 [==============================] - 0s 992us/step - loss: 9.9467 - mse: 6.1379
Epoch 7/20
13/13 [==============================] - 0s 992us/step - loss: 9.7712 - mse: 5.9774
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 9.8229 - mse: 6.0441
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 9.8820 - mse: 6.1182
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 9.7798 - mse: 6.0309

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.5190 - mse: 7.3910
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 8.2157 - mse: 7.0878
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6038 - mse: 6.4757
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.9053 - mse: 6.7773
Epoch 5/20
13/13 [==============================] - 0s 992us/step - loss: 7.1797 - mse: 6.0522
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.3916 - mse: 6.2646
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.3213 - mse: 6.1951
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.1726 - mse: 5.0473
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1432 - mse: 6.0188
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6885 - mse: 6.5651
Epoch 

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.8390 - mse: 7.5267
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 7.8900 - mse: 6.5793
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 7.8673 - mse: 6.5580
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.8072 - mse: 6.4994
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.5030 - mse: 6.1967
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.2874 - mse: 5.9826
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.4503 - mse: 6.1470
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 7.3995 - mse: 6.0977
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.9774 - mse: 5.6771
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 7.4278 - mse: 6.1291
Epoch 11

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.1582 - mse: 7.3815
Epoch 2/20
13/13 [==============================] - 0s 992us/step - loss: 8.5533 - mse: 7.7794
Epoch 3/20
13/13 [==============================] - 0s 992us/step - loss: 8.0878 - mse: 7.3152
Epoch 4/20
13/13 [==============================] - 0s 992us/step - loss: 6.9978 - mse: 6.2260
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.2652 - mse: 6.4941
Epoch 6/20
13/13 [==============================] - 0s 951us/step - loss: 6.8414 - mse: 6.0715
Epoch 7/20
13/13 [==============================] - 0s 992us/step - loss: 7.1670 - mse: 6.3989
Epoch 8/20
13/13 [==============================] - 0s 951us/step - loss: 6.9192 - mse: 6.1531
Epoch 9/20
13/13 [==============================] - 0s 909us/step - loss: 7.0361 - mse: 6.2724
Epoch 10/20
13/13 [==============================] - 0s 951us/step - loss: 6.2612 - mse:

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 909us/step - loss: 7.6296 - mse: 7.3954
Epoch 2/20
13/13 [==============================] - 0s 868us/step - loss: 6.9477 - mse: 6.7132
Epoch 3/20
13/13 [==============================] - 0s 951us/step - loss: 6.8167 - mse: 6.5813
Epoch 4/20
13/13 [==============================] - 0s 868us/step - loss: 6.4638 - mse: 6.2274
Epoch 5/20
13/13 [==============================] - 0s 951us/step - loss: 7.0156 - mse: 6.7784
Epoch 6/20
13/13 [==============================] - 0s 868us/step - loss: 6.2631 - mse: 6.0253
Epoch 7/20
13/13 [==============================] - 0s 992us/step - loss: 5.8946 - mse: 5.6564
Epoch 8/20
13/13 [==============================] - 0s 909us/step - loss: 6.1173 - mse: 5.8790
Epoch 9/20
13/13 [==============================] - 0s 909us/step - loss: 5.8250 - mse: 5.5865
Epoch 10/20
13/13 [==============================] - 0s 868us/step - loss: 6.2365 - 

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2']



---



1_2_3

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 909us/step - loss: 7.7887 - mse: 7.3471
Epoch 2/20
13/13 [==============================] - 0s 909us/step - loss: 6.9721 - mse: 6.5311
Epoch 3/20
13/13 [==============================] - 0s 868us/step - loss: 6.8520 - mse: 6.4117
Epoch 4/20
13/13 [==============================] - 0s 827us/step - loss: 6.7194 - mse: 6.2797
Epoch 5/20
13/13 [==============================] - 0s 827us/step - loss: 6.1647 - mse: 5.7256
Epoch 6/20
13/13 [==============================] - 0s 827us/step - loss: 6.1822 - mse: 5.7438
Epoch 7/20
13/13 [==============================] - 0s 744us/step - loss: 7.2450 - mse: 6.8073
Epoch 8/20
13/13 [==============================] - 0s 827us/step - loss: 6.1507 - mse: 5.7136
Epoch 9/20
13/13 [==============================] - 0s 827us/step - loss: 6.6813 - mse: 6.2449
Epoch 10/20
13/13 [==============================] - 0s 827us/step - loss: 5.9639 - 

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 9.2514 - mse: 7.6571
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 9.1338 - mse: 7.5405
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 8.6874 - mse: 7.0948
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 7.9588 - mse: 6.3668
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 7.8248 - mse: 6.2337
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.6841 - mse: 6.0940
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.7418 - mse: 6.1528
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 8.0264 - mse: 6.4387
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 7.2799 - mse: 5.6936
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 8.2269 - mse: 6.6420
Epoch 11

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 2ms/step - loss: 13.8878 - mse: 7.5553
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 12.8050 - mse: 6.5094
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 12.5089 - mse: 6.2497
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 12.1094 - mse: 5.8890
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 12.8896 - mse: 6.7097
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 12.2628 - mse: 6.1251
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 12.4261 - mse: 6.3309
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 11.9898 - mse: 5.9374
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 11.6590 - mse: 5.6497
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 11.3407 - mse: 5.375

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 8.3965 - mse: 7.7289
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 7.9047 - mse: 7.2377
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8047 - mse: 6.1374
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8510 - mse: 6.1833
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 6.8656 - mse: 6.1977
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 7.2662 - mse: 6.5984
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.2426 - mse: 6.5752
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3287 - mse: 5.6618
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.5475 - mse: 5.8813
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 6.9982 - mse: 6.3327
Epoch 11

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 22.5811 - mse: 7.5639
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 22.6001 - mse: 7.7232
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 21.6002 - mse: 6.8581
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 21.1143 - mse: 6.5049
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 21.1982 - mse: 6.7229
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 20.5515 - mse: 6.2125
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 20.1009 - mse: 5.8993
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 19.9776 - mse: 5.9146
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 20.4352 - mse: 6.5107
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 19.4074 - mse: 5.621

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 7.6837 - mse: 7.2895
Epoch 2/20
13/13 [==============================] - 0s 909us/step - loss: 6.3689 - mse: 5.9753
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2825 - mse: 5.8895
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 6.7729 - mse: 6.3805
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 5.8443 - mse: 5.4526
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.4022 - mse: 6.0110
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.0247 - mse: 5.6341
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.1842 - mse: 5.7942
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.1788 - mse: 5.7893
Epoch 10/20
13/13 [==============================] - 0s 951us/step - loss: 6.5457 - mse: 6.1569
Epoc

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 14.5660 - mse: 7.5314
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 14.1102 - mse: 7.1026
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 13.5788 - mse: 6.5944
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 13.0964 - mse: 6.1347
Epoch 5/20
13/13 [==============================] - 0s 2ms/step - loss: 12.4325 - mse: 5.4950
Epoch 6/20
13/13 [==============================] - 0s 2ms/step - loss: 12.8305 - mse: 5.9180
Epoch 7/20
13/13 [==============================] - 0s 2ms/step - loss: 12.5393 - mse: 5.6527
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 12.8013 - mse: 5.9409
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 13.1988 - mse: 6.3651
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 12.4357 - mse: 5.628

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 909us/step - loss: 7.2118 - mse: 7.1853
Epoch 2/20
13/13 [==============================] - 0s 868us/step - loss: 7.2202 - mse: 7.1936
Epoch 3/20
13/13 [==============================] - 0s 868us/step - loss: 6.5790 - mse: 6.5523
Epoch 4/20
13/13 [==============================] - 0s 909us/step - loss: 6.3820 - mse: 6.3553
Epoch 5/20
13/13 [==============================] - 0s 868us/step - loss: 5.6718 - mse: 5.6450
Epoch 6/20
13/13 [==============================] - 0s 785us/step - loss: 5.8485 - mse: 5.8217
Epoch 7/20
13/13 [==============================] - 0s 785us/step - loss: 5.8585 - mse: 5.8317
Epoch 8/20
13/13 [==============================] - 0s 909us/step - loss: 5.7970 - mse: 5.7701
Epoch 9/20
13/13 [==============================] - 0s 827us/step - loss: 5.9270 - mse: 5.9001
Epoch 10/20
13/13 [==============================] - 0s 827us/step - loss: 5.6094 - 

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 909us/step - loss: 8.7447 - mse: 7.5800
Epoch 2/20
13/13 [==============================] - 0s 785us/step - loss: 8.9233 - mse: 7.7621
Epoch 3/20
13/13 [==============================] - 0s 827us/step - loss: 7.0262 - mse: 5.8650
Epoch 4/20
13/13 [==============================] - 0s 827us/step - loss: 7.0882 - mse: 5.9264
Epoch 5/20
13/13 [==============================] - 0s 868us/step - loss: 6.8545 - mse: 5.6940
Epoch 6/20
13/13 [==============================] - 0s 868us/step - loss: 7.7389 - mse: 6.5811
Epoch 7/20
13/13 [==============================] - 0s 827us/step - loss: 7.3756 - mse: 6.2216
Epoch 8/20
13/13 [==============================] - 0s 909us/step - loss: 7.1180 - mse: 5.9681
Epoch 9/20
13/13 [==============================] - 0s 868us/step - loss: 7.1913 - mse: 6.0461
Epoch 10/20
13/13 [==============================] - 0s 868us/step - loss: 7.1459 - 

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 0s 909us/step - loss: 7.4271 - mse: 7.4271
Epoch 2/20
13/13 [==============================] - 0s 827us/step - loss: 6.8475 - mse: 6.8475
Epoch 3/20
13/13 [==============================] - 0s 785us/step - loss: 7.1817 - mse: 7.1817
Epoch 4/20
13/13 [==============================] - 0s 868us/step - loss: 6.9521 - mse: 6.9521
Epoch 5/20
13/13 [==============================] - 0s 827us/step - loss: 6.3705 - mse: 6.3705
Epoch 6/20
13/13 [==============================] - 0s 868us/step - loss: 5.6693 - mse: 5.6693
Epoch 7/20
13/13 [==============================] - 0s 827us/step - loss: 6.0655 - mse: 6.0655
Epoch 8/20
13/13 [==============================] - 0s 827us/step - loss: 6.4836 - mse: 6.4836
Epoch 9/20
13/13 [==============================] - 0s 909us/step - loss: 5.7367 - mse: 5.7367
Epoch 10/20
13/13 [==============================] - 0s 909us/step - loss: 6.3257 - 

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3']



---



1_2_3_4

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3]).join(datalab4[selected_features_ann_4])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 951us/step - loss: 8.5554 - mse: 7.6975
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 7.5641 - mse: 6.7081
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 8.0664 - mse: 7.2116
Epoch 4/20
13/13 [==============================] - 0s 951us/step - loss: 7.0549 - mse: 6.2013
Epoch 5/20
13/13 [==============================] - 0s 992us/step - loss: 7.4300 - mse: 6.5778
Epoch 6/20
13/13 [==============================] - 0s 992us/step - loss: 7.0412 - mse: 6.1910
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 7.1211 - mse: 6.2733
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.6806 - mse: 5.8355
Epoch 9/20
13/13 [==============================] - 0s 992us/step - loss: 6.3232 - mse: 5.4812
Epoch 10/20
13/13 [==============================] - 0s 992us/step - loss: 6.6560 - mse: 5.8

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3]).join(datalab4[selected_features_ann_4])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 7.8710 - mse: 7.8509
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 6.9052 - mse: 6.8850
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2833 - mse: 6.2631
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 6.2133 - mse: 6.1931
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 6.4520 - mse: 6.4318
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 6.0929 - mse: 6.0726
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3041 - mse: 6.2838
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3618 - mse: 6.3414
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 6.3727 - mse: 6.3523
Epoch 10/20
13/13 [==============================] - 0s 992us/step - loss: 5.5760 - mse: 5.5556
Epoch 

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3]).join(datalab4[selected_features_ann_4])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 909us/step - loss: 8.1057 - mse: 7.7863
Epoch 2/20
13/13 [==============================] - 0s 827us/step - loss: 6.9230 - mse: 6.6032
Epoch 3/20
13/13 [==============================] - 0s 785us/step - loss: 6.9711 - mse: 6.6505
Epoch 4/20
13/13 [==============================] - 0s 744us/step - loss: 7.2476 - mse: 6.9262
Epoch 5/20
13/13 [==============================] - 0s 785us/step - loss: 5.8638 - mse: 5.5420
Epoch 6/20
13/13 [==============================] - 0s 703us/step - loss: 6.3574 - mse: 6.0352
Epoch 7/20
13/13 [==============================] - 0s 785us/step - loss: 7.1795 - mse: 6.8571
Epoch 8/20
13/13 [==============================] - 0s 827us/step - loss: 6.2069 - mse: 5.8844
Epoch 9/20
13/13 [==============================] - 0s 827us/step - loss: 6.1597 - mse: 5.8372
Epoch 10/20
13/13 [==============================] - 0s 744us/step - loss: 6.8324 - 

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3]).join(datalab4[selected_features_ann_4])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 868us/step - loss: 7.3931 - mse: 7.2677
Epoch 2/20
13/13 [==============================] - 0s 744us/step - loss: 6.2499 - mse: 6.1243
Epoch 3/20
13/13 [==============================] - 0s 785us/step - loss: 5.6759 - mse: 5.5502
Epoch 4/20
13/13 [==============================] - 0s 744us/step - loss: 5.3569 - mse: 5.2311
Epoch 5/20
13/13 [==============================] - 0s 744us/step - loss: 6.3634 - mse: 6.2375
Epoch 6/20
13/13 [==============================] - 0s 785us/step - loss: 6.7002 - mse: 6.5743
Epoch 7/20
13/13 [==============================] - 0s 909us/step - loss: 5.8632 - mse: 5.7373
Epoch 8/20
13/13 [==============================] - 0s 744us/step - loss: 6.0783 - mse: 5.9523
Epoch 9/20
13/13 [==============================] - 0s 827us/step - loss: 6.5609 - mse: 6.4350
Epoch 10/20
13/13 [==============================] - 0s 827us/step - loss: 6.1525 - 

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3]).join(datalab4[selected_features_ann_4])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 1ms/step - loss: 10.5571 - mse: 7.5986
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 10.8478 - mse: 7.9028
Epoch 3/20
13/13 [==============================] - 0s 1ms/step - loss: 10.0239 - mse: 7.0905
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 10.1964 - mse: 7.2731
Epoch 5/20
13/13 [==============================] - 0s 1ms/step - loss: 9.2959 - mse: 6.3830
Epoch 6/20
13/13 [==============================] - 0s 1ms/step - loss: 8.5410 - mse: 5.6394
Epoch 7/20
13/13 [==============================] - 0s 1ms/step - loss: 8.7363 - mse: 5.8466
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 8.9333 - mse: 6.0561
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 8.5497 - mse: 5.6853
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 8.1388 - mse: 5.2874
Epoc

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3]).join(datalab4[selected_features_ann_4])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 951us/step - loss: 8.4631 - mse: 7.4663
Epoch 2/20
13/13 [==============================] - 0s 1ms/step - loss: 7.7122 - mse: 6.7159
Epoch 3/20
13/13 [==============================] - 0s 951us/step - loss: 7.3698 - mse: 6.3732
Epoch 4/20
13/13 [==============================] - 0s 951us/step - loss: 7.1783 - mse: 6.1816
Epoch 5/20
13/13 [==============================] - 0s 868us/step - loss: 7.2082 - mse: 6.2117
Epoch 6/20
13/13 [==============================] - 0s 868us/step - loss: 7.1852 - mse: 6.1892
Epoch 7/20
13/13 [==============================] - 0s 868us/step - loss: 7.4902 - mse: 6.4949
Epoch 8/20
13/13 [==============================] - 0s 827us/step - loss: 7.1549 - mse: 6.1604
Epoch 9/20
13/13 [==============================] - 0s 868us/step - loss: 7.1954 - mse: 6.2018
Epoch 10/20
13/13 [==============================] - 0s 868us/step - loss: 7.5318 - ms

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3]).join(datalab4[selected_features_ann_4])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 827us/step - loss: 10.9217 - mse: 7.7862
Epoch 2/20
13/13 [==============================] - 0s 663us/step - loss: 11.2953 - mse: 8.1692
Epoch 3/20
13/13 [==============================] - 0s 747us/step - loss: 9.7220 - mse: 6.6046
Epoch 4/20
13/13 [==============================] - 0s 785us/step - loss: 10.2300 - mse: 7.1206
Epoch 5/20
13/13 [==============================] - 0s 785us/step - loss: 8.9030 - mse: 5.8008
Epoch 6/20
13/13 [==============================] - 0s 827us/step - loss: 9.1473 - mse: 6.0515
Epoch 7/20
13/13 [==============================] - 0s 870us/step - loss: 8.8363 - mse: 5.7469
Epoch 8/20
13/13 [==============================] - 0s 785us/step - loss: 9.2853 - mse: 6.2026
Epoch 9/20
13/13 [==============================] - 0s 827us/step - loss: 9.1759 - mse: 6.1005
Epoch 10/20
13/13 [==============================] - 0s 909us/step - loss: 9.4803

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3]).join(datalab4[selected_features_ann_4])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 868us/step - loss: 8.2518 - mse: 8.1725
Epoch 2/20
13/13 [==============================] - 0s 827us/step - loss: 7.1259 - mse: 7.0466
Epoch 3/20
13/13 [==============================] - 0s 785us/step - loss: 6.2958 - mse: 6.2163
Epoch 4/20
13/13 [==============================] - 0s 827us/step - loss: 5.8688 - mse: 5.7892
Epoch 5/20
13/13 [==============================] - 0s 785us/step - loss: 6.0616 - mse: 5.9819
Epoch 6/20
13/13 [==============================] - 0s 829us/step - loss: 6.4059 - mse: 6.3261
Epoch 7/20
13/13 [==============================] - 0s 912us/step - loss: 6.4462 - mse: 6.3663
Epoch 8/20
13/13 [==============================] - 0s 785us/step - loss: 4.8691 - mse: 4.7892
Epoch 9/20
13/13 [==============================] - 0s 785us/step - loss: 6.5064 - mse: 6.4265
Epoch 10/20
13/13 [==============================] - 0s 827us/step - loss: 5.7907 - 

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3]).join(datalab4[selected_features_ann_4])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 910us/step - loss: 11.3160 - mse: 8.3123
Epoch 2/20
13/13 [==============================] - 0s 951us/step - loss: 11.0893 - mse: 8.0965
Epoch 3/20
13/13 [==============================] - 0s 909us/step - loss: 10.6167 - mse: 7.6334
Epoch 4/20
13/13 [==============================] - 0s 868us/step - loss: 9.9561 - mse: 6.9814
Epoch 5/20
13/13 [==============================] - 0s 909us/step - loss: 10.2359 - mse: 7.2690
Epoch 6/20
13/13 [==============================] - 0s 951us/step - loss: 9.6038 - mse: 6.6439
Epoch 7/20
13/13 [==============================] - 0s 992us/step - loss: 9.5123 - mse: 6.5596
Epoch 8/20
13/13 [==============================] - 0s 951us/step - loss: 9.5974 - mse: 6.6522
Epoch 9/20
13/13 [==============================] - 0s 909us/step - loss: 8.5534 - mse: 5.6160
Epoch 10/20
13/13 [==============================] - 0s 992us/step - loss: 8.949

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']

In [ ]:
dataset = datalab1[[TARGET] + selected_features_ann_1].join(datalab2[selected_features_ann_2]).join(datalab3[selected_features_ann_3]).join(datalab4[selected_features_ann_4])
run_process(dataset,rand_cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/20
13/13 [==============================] - 1s 992us/step - loss: 9.7804 - mse: 7.3625
Epoch 2/20
13/13 [==============================] - 0s 992us/step - loss: 8.9935 - mse: 6.5839
Epoch 3/20
13/13 [==============================] - 0s 951us/step - loss: 9.1002 - mse: 6.6971
Epoch 4/20
13/13 [==============================] - 0s 1ms/step - loss: 8.8673 - mse: 6.4717
Epoch 5/20
13/13 [==============================] - 0s 992us/step - loss: 8.0714 - mse: 5.6844
Epoch 6/20
13/13 [==============================] - 0s 992us/step - loss: 8.4671 - mse: 6.0896
Epoch 7/20
13/13 [==============================] - 0s 992us/step - loss: 8.5579 - mse: 6.1905
Epoch 8/20
13/13 [==============================] - 0s 1ms/step - loss: 8.2968 - mse: 5.9401
Epoch 9/20
13/13 [==============================] - 0s 1ms/step - loss: 8.1501 - mse: 5.8041
Epoch 10/20
13/13 [==============================] - 0s 1ms/step - loss: 8.2848 - mse: 5.9

['actq3_lab1',
 'actq1_lab1',
 'mean(qmsr$_lab1)',
 'rtr_lab1',
 'cer_lab1',
 'actq3_lab2',
 'actq1_lab2',
 'mean(qavt$_lab2)',
 'avgtime_lab2',
 'rt_lab2',
 'rt_lab3',
 'actq1_lab3',
 'ut_lab3',
 'mean(qact$_lab3)',
 'cer_lab3',
 'rt_lab4',
 'cer_lab4',
 'err_lab4',
 'actq1_lab4',
 'mean(qc$_lab4)']